# Loading MOVER dataset

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime

# MOVER (about 39,,685 patients, 64,354 cases)
# Define the path to the INSPIRE v2 dataset
input_path = 'mover/EPIC_EMR'

# Load the source tables within INSPIRE v2 into dataframes
df_info = pd.read_csv(f'{input_path}/patient_information.csv')         # Load information data
df_hist = pd.read_csv(f'{input_path}/patient_history.csv')              # Load history data
df_medi = pd.read_csv(f'{input_path}/patient_medications.csv')       # Load medications data
df_diag = pd.read_csv(f'{input_path}/patient_visit.csv')          # Load visit(diagnosis) data
df_events = pd.read_csv(f'{input_path}/patient_procedure_events.csv')          # Load vitals data
df_labs = pd.read_csv(f'{input_path}/patient_labs.csv')       # Load labs data
df_devices = pd.read_csv(f'{input_path}/patient_lda.csv') 
df_comp = pd.read_csv(f'{input_path}/patient_post_op_complications.csv') 

# Display the number of records in each dataset
print(f'Size of the tables: information {len(df_info)}, history {len(df_hist)}, visit(diagnosis) {len(df_diag)}, labs {len(df_labs)}, medications {len(df_medi)}, events {len(df_events)}, devices {len(df_devices)}, complication {len(df_comp)}')

# Combine all the subject_ids from the loaded datasets
subject_ids = df_diag['mrn'].tolist() + df_labs['MRN'].tolist() + df_medi['MRN'].tolist() + df_info['MRN'].tolist() + df_hist['mrn'].tolist() + df_events['MRN'].tolist() + df_devices['MRN'].tolist() + df_comp['MRN'].tolist()

# Display the total unique subjects present in the combined dataset
print(f'total subjects in MOVER dataset: {len(np.unique(subject_ids))}')
print(f"total subjects in patient_information.csv: {len(np.unique(df_info['MRN']))}")

/tmp/ipykernel_1911732/3885464592.py:13: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_medi = pd.read_csv(f'{input_path}/patient_medications.csv')       # Load medications data


Size of the tables: information 65728, history 970741, visit(diagnosis) 219257, labs 29079344, medications 27961524, events 640223, devices 22965, complication 203945
total subjects in MOVER dataset: 44593
total subjects in patient_information.csv: 39685


In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime

# MOVER (about 39,,685 patients, 64,354 cases)
# Define the path to the INSPIRE v2 dataset
input_path = 'mover/EPIC_EMR/parquet'

# Load the source tables within INSPIRE v2 into dataframes
df_info = pd.read_parquet(f'{input_path}/patient_information.parquet')         # Load information data
df_hist = pd.read_parquet(f'{input_path}/patient_history.parquet')              # Load history data
df_medi = pd.read_parquet(f'{input_path}/patient_medications.parquet')       # Load medications data
df_diag = pd.read_parquet(f'{input_path}/patient_visit.parquet')          # Load visit(diagnosis) data
df_events = pd.read_parquet(f'{input_path}/patient_procedure_events.parquet')          # Load vitals data
df_labs = pd.read_parquet(f'{input_path}/patient_labs.parquet')       # Load labs data
df_devices = pd.read_parquet(f'{input_path}/patient_lda.parquet') 
df_comp = pd.read_parquet(f'{input_path}/patient_comp.parquet') 

# Display the number of records in each dataset
print(f'Size of the tables: information {len(df_info)}, history {len(df_hist)}, visit(diagnosis) {len(df_diag)}, labs {len(df_labs)}, medications {len(df_medi)}, events {len(df_events)}, devices {len(df_devices)}, complication {len(df_comp)}')

# Combine all the subject_ids from the loaded datasets
subject_ids = df_diag['MRN'].tolist() + df_labs['MRN'].tolist() + df_medi['MRN'].tolist() + df_info['MRN'].tolist() + df_hist['MRN'].tolist() + df_events['MRN'].tolist() + df_devices['MRN'].tolist() + df_comp['MRN'].tolist()

# Display the total unique subjects present in the combined dataset
print(f'total subjects in MOVER dataset: {len(np.unique(subject_ids))}')
print(f"total subjects in patient_information.parquet: {len(np.unique(df_info['MRN']))}")

Size of the tables: information 65728, history 970741, visit(diagnosis) 219257, labs 29079344, medications 43091132, events 640223, devices 22965, complication 203945
total subjects in MOVER dataset: 44593
total subjects in patient_information.parquet: 39685


## preprocessing

### Others

In [776]:
# patient_information table
input_path = 'mover/EPIC_EMR'
df_info = pd.read_csv(f'{input_path}/patient_information.csv')         # Load information data

# Convert ID (hexadeximal string or decimal string) into decimal
def convert_to_int(value):
    import re
    try:
        # Check if the value is a string
        if isinstance(value, str):
            # Remove any whitespace for accurate checking
            value = value.strip()
            # Check if the string is a valid hexadecimal characters
            if re.search(r'^[0-9a-fA-F]+$', value):
                return int(value, 16)            
            # Check if the string is in scientific notation
            elif 'E' in value or 'e' in value:
                return int(float(value))
            else:
                return None
    except ValueError:
        # If the conversion fails
        return None

# Apply the function to the 'MRN' column
#df_info['MRN'] = df_info['MRN'].apply(convert_to_int)

# Apply the function to the 'LOG_ID' column
#df_info['LOG_ID'] = df_info['LOG_ID'].apply(convert_to_int)

# Convert the datetime in string into datetime64[ns] type
df_info['HOSP_ADMSN_TIME'] = pd.to_datetime(df_info['HOSP_ADMSN_TIME'], format='%m/%d/%y %H:%M', errors='raise')
df_info['HOSP_DISCH_TIME'] = pd.to_datetime(df_info['HOSP_DISCH_TIME'], format='%m/%d/%y %H:%M', errors='raise')
df_info['SURGERY_DATE'] = pd.to_datetime(df_info['SURGERY_DATE'], format='%m/%d/%y %H:%M', errors='raise')
df_info['IN_OR_DTTM'] = pd.to_datetime(df_info['IN_OR_DTTM'], format='%m/%d/%y %H:%M', errors='raise')
df_info['OUT_OR_DTTM'] = pd.to_datetime(df_info['OUT_OR_DTTM'], format='%m/%d/%y %H:%M', errors='raise')
df_info['AN_START_DATETIME'] = pd.to_datetime(df_info['AN_START_DATETIME'], format='%m/%d/%y %H:%M', errors='raise')
df_info['AN_STOP_DATETIME'] = pd.to_datetime(df_info['AN_STOP_DATETIME'], format='%m/%d/%y %H:%M', errors='raise')

# Split the string into feet and inches using vectorized string operations
height_split = df_info['HEIGHT'].str.extract(r"(?P<Feet>\d+)' (?P<Inches>\d+)")
# Convert feet and inches to numeric (float) and then to cm
df_info['HEIGHT'] = height_split['Feet'].astype(float) * 30.48 + height_split['Inches'].astype(float) * 2.54

# Convert weight in grams to kilograms
df_info['WEIGHT'] = df_info['WEIGHT'].astype(float) / 10 

# Rename the column name as it is described in documentation
df_info.rename(columns={'ICU_ADMIN_FLAG': 'ICU_ADMIN'}, inplace=True)
#df_info['ICU_ADMIN'] = df_info['ICU_ADMIN'].map({'Yes':True, 'No': False}

# Type conversion for efficiency
df_info = df_info.astype({'DISCH_DISP_C': 'Int8', 'LOS': 'Int16', 'BIRTH_DATE': 'Int8', 'ASA_RATING_C': 'Int8'})

print(df_info.info())

df_info.to_parquet(f'{input_path}/parquet/patient_information.parquet', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65728 entries, 0 to 65727
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   LOG_ID                65728 non-null  object        
 1   MRN                   65728 non-null  object        
 2   DISCH_DISP_C          65721 non-null  Int8          
 3   DISCH_DISP            65721 non-null  object        
 4   HOSP_ADMSN_TIME       65728 non-null  datetime64[ns]
 5   HOSP_DISCH_TIME       65714 non-null  datetime64[ns]
 6   LOS                   65714 non-null  Int16         
 7   ICU_ADMIN             65728 non-null  object        
 8   SURGERY_DATE          65728 non-null  datetime64[ns]
 9   BIRTH_DATE            65728 non-null  Int8          
 10  HEIGHT                52567 non-null  float64       
 11  WEIGHT                63319 non-null  float64       
 12  SEX                   65728 non-null  object        
 13  PRIMARY_ANES_TYP

In [124]:
### patient_labs table ###
# Load Athena concept, concept_relationship table
df_vocab = pd.read_csv(f'vocab/VOCABULARY.csv', sep='\t', on_bad_lines='error')
df_concept_rel = pd.read_csv(f'vocab/CONCEPT_RELATIONSHIP.csv', sep='\t', on_bad_lines='error')
df_concept = pd.read_csv(f'vocab/CONCEPT.csv', sep='\t')

# Filter concepts to only include 'Unit' domain and standard concepts
unit_concepts = df_concept[(df_concept['domain_id']=='Unit') & (df_concept['standard_concept']=='S')]
# Convert all concept codes to lower case for uniformity
unit_concepts['concept_code'] = unit_concepts['concept_code'].str.lower()

# Make a copy of the labs dataframe for manipulation
df = df_labs.copy()
# Convert all measurement units in the dataframe to lower case for consistency
df['Measurement Units'] = df['Measurement Units'].str.lower()
# Remove duplicate entries based on 'MRN' and 'Measurement Units', keep unique combinations
df1 = df.drop_duplicates(subset=['MRN', 'Measurement Units'])['Measurement Units']
# Calculate the frequency percentage of each unique measurement unit across all unique MRNs
df1 = (df1.value_counts() / len(df['MRN'].unique()) * 100).to_frame()
df1.reset_index(inplace=True)     

# Merge the frequency dataframe with the unit concepts based on concept codes
df1 = df1.merge(unit_concepts[['concept_id', 'concept_code']], left_on='Measurement Units', right_on='concept_code', how='left')
# Export the merged dataframe to a CSV file, omitting the index
df1.to_csv('results/measurement_unit_counts.csv', index=False)

# Load the manually mapped measurement units file into a dataframe
mapped_path = 'results/labeled'
unit_mapped = pd.read_csv(f'{mapped_path}/labs_units.csv')

/tmp/ipykernel_1216506/4143165479.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unit_concepts['concept_code'] = unit_concepts['concept_code'].str.lower()


In [144]:
# Extract LOINC codes and their corresponding concept IDs
loinc_concepts = df_concept.loc[df_concept['vocabulary_id'] == 'LOINC', ['concept_code', 'concept_id']]
# Rename 'concept_code' column for clarity
loinc_concepts.rename(columns={'concept_code': 'Lab Code'}, inplace=True)
# Merge lab data with LOINC concepts to add concept IDs
df_labs = df_labs.merge(loinc_concepts, on='Lab Code', how='left')

# Map unit concept IDs from the unit_mapped dataframe to df_labs
df_labs['unit_concept_id'] = df.merge(unit_mapped[['Measurement Units', 'concept_id']], on='Measurement Units', how='left')['concept_id']
# Convert the 'unit_concept_id' to nullable integers to accommodate potential missing values
df_labs['unit_concept_id'] = df_labs['unit_concept_id'].astype('Int64')

# Drop the 'ENC_TYPE_NM' column from the labs dataframe
df_labs.drop(columns=['ENC_TYPE_NM'], inplace=True)

# Convert 'Collection Datetime' to datetime format, raising errors for any conversion issues
df_labs['Collection Datetime'] = pd.to_datetime(df_labs['Collection Datetime'], format='%Y-%m-%d %H:%M:%S', errors='raise')

# Change data type of 'Observation Value' to float
df_labs = df_labs.astype({'Observation Value': 'float'})

# Display dataframe information, useful for verifying data types and non-null counts
print(df_labs.info())

# Define the save path for the processed dataframe
save_path = 'mover/EPIC_EMR/parquet'
# Save the labs dataframe to a parquet file
df_labs.to_parquet(f'{save_path}/patient_labs.parquet', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29079344 entries, 0 to 29079343
Data columns (total 11 columns):
 #   Column               Dtype         
---  ------               -----         
 0   LOG_ID               object        
 1   MRN                  object        
 2   Lab Code             object        
 3   Lab Name             object        
 4   Observation Value    float64       
 5   Measurement Units    object        
 6   Reference Range      object        
 7   Abnormal Flag        object        
 8   Collection Datetime  datetime64[ns]
 9   concept_id           int64         
 10  unit_concept_id      Int64         
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 2.4+ GB


In [151]:
### patient_history table ###
# Select ICD9CM codes and corresponding concept IDs from df_concept
icd9cm_concepts = df_concept.loc[df_concept['vocabulary_id'] == 'ICD9CM', ['concept_code', 'concept_id']]

# Map concept IDs to patient_history table based on diagnosis codes
df_hist['concept_id'] = df_hist.merge(icd9cm_concepts, left_on='diagnosis_code', right_on='concept_code', how='left')['concept_id'].astype('Int64')

# Standardize column name for 'MRN' to match other tables
df_hist.rename(columns={'mrn': 'MRN'}, inplace=True)

# Save the patient history data as a parquet file
df_hist.to_parquet(f'{save_path}/patient_hist.parquet', index=False)

In [174]:
### patient_visit table ###
# Map concept IDs to patient_history table based on diagnosis codes
df_visit['concept_id'] = df_visit.merge(icd9cm_concepts, left_on='diagnosis_code', right_on='concept_code', how='left')['concept_id'].astype('Int64')

# Standardize column name for 'MRN' to match other tables
df_visit.rename(columns={'mrn':'MRN'}, inplace=True)

# Drop same rows
df_visit.drop_duplicates(inplace=True)

# Save the visit table to a parquet file
df_visit.to_parquet(f'{save_path}/patient_visit.parquet', index=False)

In [179]:
### patient_procedure_events table ###
# Convert the 'EVENT_TIME' column to datetime64 format
df_events['EVENT_TIME'] = pd.to_datetime(df_events['EVENT_TIME'], format='%m/%d/%y %H:%M', errors='raise')

# Remove the 'NOTE_TEXT' column from the dataframe as it is not needed for further analysis
df_events.drop(columns='NOTE_TEXT', inplace=True)

# Save the history table to a parquet file
df_events.to_parquet(f'{save_path}/patient_procedure_events.parquet', index=False)

In [691]:
### patient_medication table ###
mapped_path = 'results/labeled'
# Load the mapped medication table from a CSV file
medi_mapped = pd.read_csv(f'{mapped_path}/mover_medi_mapped.csv')

# Merge the medication mapping onto the medication table based on medication names
df_medi = df_medi.merge(medi_mapped, on='MEDICATION_NM', how='left')

# Load the manually mapped DOSE_UNIT_NM file into a dataframe
unit_mapped = pd.read_csv(f'{mapped_path}/mover_medi_unit_mapped.csv')
unit_mapped.rename(columns={'concept_id':'unit_concept_id'}, inplace=True)

# Merge the dose unit mapping onto the medication table based on medication names
df_medi = df_medi.merge(unit_mapped[['DOSE_UNIT_NM', 'concept_id']], on='DOSE_UNIT_NM', how='left')

# Convert dates to datetime format
df_medi['ORDERING_DATE'] = pd.to_datetime(df_medi['ORDERING_DATE'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df_medi['START_DATE'] = pd.to_datetime(df_medi['START_DATE'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df_medi['END_DATE'] = pd.to_datetime(df_medi['END_DATE'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df_medi['MED_ACTION_TIME'] = pd.to_datetime(df_medi['MED_ACTION_TIME'], format='%Y-%m-%d %H:%M:%S', errors='raise')

# Convert ADMIN_SIG to float type
df_medi['ADMIN_SIG'] = df_medi['ADMIN_SIG'].replace('0 NULL', '0')
df_medi['ADMIN_SIG'] = df_medi['ADMIN_SIG'].astype('float')
#df_medi['ADMIN_SIG'] = pd.to_numeric(df_medi['ADMIN_SIG'], errors='coerce')

df_medi['concept_id'] = df_medi['concept_id'].astype('Int64')
df_medi['unit_concept_id'] = df_medi['unit_concept_id'].astype('Int64')

# Drop unnecessary columns that are not used in further study
df_medi.drop(columns=['ENC_TYPE_C', 'ENC_TYPE_NM', 'ORDER_STATUS_NM', 'MAR_ACTION_NM'], inplace=True)

print(df_medi.info())

# Save the medication table to a parquet file
df_medi.to_parquet(f'{save_path}/patient_medications.parquet', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43091132 entries, 0 to 43091131
Data columns (total 17 columns):
 #   Column           Dtype         
---  ------           -----         
 0   LOG_ID           object        
 1   MRN              object        
 2   ORDERING_DATE    datetime64[ns]
 3   ORDER_CLASS_NM   object        
 4   MEDICATION_ID    int64         
 5   DISPLAY_NAME     object        
 6   MEDICATION_NM    object        
 7   START_DATE       datetime64[ns]
 8   END_DATE         datetime64[ns]
 9   RECORD_TYPE      object        
 10  MED_ACTION_TIME  datetime64[ns]
 11  ADMIN_SIG        float64       
 12  DOSE_UNIT_NM     object        
 13  MED_ROUTE_NM     object        
 14  ingredient       object        
 15  concept_id       Int64         
 16  unit_concept_id  Int64         
dtypes: Int64(2), datetime64[ns](4), float64(1), int64(1), object(9)
memory usage: 5.5+ GB


In [817]:
df_medi

,LOG_ID,MRN,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,END_DATE,RECORD_TYPE,MED_ACTION_TIME,ADMIN_SIG,DOSE_UNIT_NM,MED_ROUTE_NM,ingredient,concept_id,unit_concept_id
0,f51e86014fd956d3,bd51e3b61ead68a6,2020-08-04,Inpatient,4424,levothyroxine (SYNTHROID) tablet 125 mcg,LEVOTHYROXINE SODIUM 125 MCG OR TABS,2020-08-05,2020-08-13,POST-OP,2020-08-06 06:13:00,125.0,mcg,NG Tube,levothyroxine sodium,1501700,9655
1,f51e86014fd956d3,bd51e3b61ead68a6,2020-08-04,Inpatient,4424,levothyroxine (SYNTHROID) tablet 125 mcg,LEVOTHYROXINE SODIUM 125 MCG OR TABS,2020-08-05,2020-08-13,POST-OP,2020-08-09 06:30:00,125.0,mcg,NG Tube,levothyroxine sodium,1501700,9655
2,f51e86014fd956d3,bd51e3b61ead68a6,2020-08-04,Inpatient,4424,levothyroxine (SYNTHROID) tablet 125 mcg,LEVOTHYROXINE SODIUM 125 MCG OR TABS,2020-08-05,2020-08-13,POST-OP,2020-08-11 05:55:35,NaN,mcg,NG Tube,levothyroxine sodium,1501700,9655
3,f51e86014fd956d3,bd51e3b61ead68a6,2020-08-04,Inpatient,19177,atorvastatin (LIPITOR) tablet 40 mg,ATORVASTATIN CALCIUM 40 MG OR TABS,2020-08-04,2020-08-13,POST-OP,2020-08-07 18:00:00,40.0,mg,Oral,atorvastatin calcium,1545958,8576
4,f51e86014fd956d3,bd51e3b61ead68a6,2020-08-04,Inpatient,19177,atorvastatin (LIPITOR) tablet 40 mg,ATORVASTATIN CALCIUM 40 MG OR TABS,2020-08-04,2020-08-13,POST-OP,2020-08-11 05:55:35,NaN,mg,Oral,atorvastatin calcium,1545958,8576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43091127,ad73942a2269c78f,7b69742628c1211a,2022-08-28,Inpatient,7557,"sulfamethoxazole-trimethoprim (BACTRIM, SEPTRA...",SULFAMETHOXAZOLE-TRIMETHOPRIM 400-80 MG OR TABS,2022-08-29,2022-09-24,INTRA-OP,2022-09-03 23:20:43,NaN,tablet,Oral,trimethoprim,1705674,9431
43091128,ad73942a2269c78f,7b69742628c1211a,2022-08-28,Inpatient,988,benzonatate (TESSALON) capsule 100 mg,BENZONATATE 100 MG OR CAPS,2022-08-28,2022-09-24,INTRA-OP,2022-09-03 23:20:43,NaN,mg,Oral,benzonatate,1136601,8576
43091129,ad73942a2269c78f,7b69742628c1211a,2022-08-28,Inpatient,988,benzonatate (TESSALON) capsule 100 mg,BENZONATATE 100 MG OR CAPS,2022-08-28,2022-09-24,INTRA-OP,2022-09-03 23:20:43,NaN,mg,Oral,mg or caps,<NA>,8576
43091130,ad73942a2269c78f,7b69742628c1211a,2022-08-28,Inpatient,988,benzonatate (TESSALON) capsule 100 mg,BENZONATATE 100 MG OR CAPS,2022-08-28,2022-09-24,POST-OP,2022-09-19 15:15:41,NaN,mg,Oral,benzonatate,1136601,8576


In [629]:
## Calculate counts of ingredients per MRN for analysis
# Remove duplicate records for each MRN and concept_id pair, keeping only the ingredient and concept_id columns
medi_match = df_medi.drop_duplicates(subset=['MRN', 'concept_id'])[['ingredient', 'concept_id']]

# Count occurrences of each unique ingredient, convert the series to a dataframe
drug_counts = medi_match['ingredient'].value_counts().to_frame()
drug_counts.reset_index(inplace=True)
# Calculate the percentage of total unique MRNs for each ingredient
drug_counts['perc'] = (drug_counts['ingredient'] / len(df_medi['MRN'].unique())) * 100

# Remove duplicate entries of ingredients to have a unique list
medi_match = medi_match.drop_duplicates(subset=['ingredient'])
# Merge the counts and percentages back into the medi_match dataframe
medi_match = medi_match.merge(drug_counts, on='ingredient', how='left')
# Sort the dataframe by the percentage column in descending order and reset the index
medi_match = medi_match.sort_values(by='perc', ascending=False, ignore_index=True)

# Save the resulting dataframe to a CSV file
medi_match.to_csv('results/mover_medi_counts.csv', index=False)

medi_match

,ingredient,concept_id,count,perc
0,propofol,753626.0,36895.0,95.353165
1,acetaminophen,1125315.0,34695.0,89.667382
2,fentanyl citrate,1154029.0,34178.0,88.331223
3,ondansetron hcl,1000560.0,34084.0,88.088285
4,dexamethasone sodium phosphate,1518254.0,30849.0,79.727599
...,...,...,...,...
1901,verapamil,1307863.0,1.0,0.002584
1902,bss plus io soln,NaN,1.0,0.002584
1903,unit im solr,NaN,1.0,0.002584
1904,neostigmine,717136.0,1.0,0.002584


In [645]:
## Calculating counts of dose units and mapping to standard concept
dose_units = df_medi['DOSE_UNIT_NM'].value_counts()

# Make a copy of the labs dataframe for manipulation
df = df_medi.copy()
# Convert all DOSE_UNIT_NM in the dataframe to lower case for consistency
df['DOSE_UNIT_NM'] = df['DOSE_UNIT_NM'].str.lower()
# Remove duplicate entries based on 'MRN' and 'DOSE_UNIT_NM', keep unique combinations
df1 = df.drop_duplicates(subset=['MRN', 'DOSE_UNIT_NM'])['DOSE_UNIT_NM']
# Calculate the frequency percentage of each unique measurement unit across all unique MRNs
df1 = (df1.value_counts() / len(df['MRN'].unique()) * 100).to_frame()
df1.reset_index(inplace=True)     

# Merge the frequency dataframe with the unit concepts based on concept codes
df1 = df1.merge(unit_concepts[['concept_id', 'concept_code']], left_on='DOSE_UNIT_NM', right_on='concept_code', how='left')
# Export the merged dataframe to a CSV file, omitting the index
df1.to_csv('results/drug_unit_counts.csv', index=False)

# Load the manually mapped DOSE_UNIT_NM file into a dataframe
mapped_path = 'results/labeled'
unit_mapped = pd.read_csv(f'{mapped_path}/mover_medi_unit_mapped.csv')

In [695]:
### patient_lda table ###
df_lda['placement_instant'] = pd.to_datetime(df_lda['placement_instant'], format='%m/%d/%y %H:%M', errors='raise')
df_lda['removal_instant'] = pd.to_datetime(df_lda['removal_instant'], format='%m/%d/%y %H:%M', errors='raise')

# Save the lda table to a parquet file
df_lda.to_parquet(f'{save_path}/patient_lda.parquet', index=False)

In [698]:
### postoperative complications
# Save the postoperative complications table to a parquet file
df_comp.to_parquet(f'{save_path}/patient_comp.parquet', index=False)

In [ ]:
# Load the source tables within MOVER into dataframes
df_info = pd.read_csv(f'{input_path}/patient_information.csv')         # Load information data
df_hist = pd.read_csv(f'{input_path}/patient_history.csv')              # Load history data
df_medi = pd.read_csv(f'{input_path}/patient_medications.csv')       # Load medications data
df_visit = pd.read_csv(f'{input_path}/patient_visit.csv')          # Load visit(diagnosis) data
df_events = pd.read_csv(f'{input_path}/patient_procedure_events.csv')          # Load vitals data
df_labs = pd.read_csv(f'{input_path}/patient_labs.csv')       # Load labs data
df_lda = pd.read_csv(f'{input_path}/patient_lda.csv') 
df_comp = pd.read_csv(f'{input_path}/patient_post_op_complications.csv') 

### measurement

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime

def read_csv(filename, portion = 0.1):
    # Determine the number of lines
    num_lines = sum(1 for l in open(filename))

    # Calculate 10% of the total lines
    sample_size = int(portion * num_lines)
    
    print(f'Sample {portion*100} % of {filename} ({sample_size} lines)')

    # Create a list of rows to skip
    import random
    skip_rows = random.sample(range(1, num_lines), num_lines - sample_size)

    # Return the CSV, skipping the rows in skip_rows
    col_names = ['LOG_ID', 'MRN', 'FLO_NAME', 'FLO_DISPLAY_NAME', 'RECORD_TYPE', 'RECORD_TIME', 'MEAS_VALUE', 'UNITS']
    
    # check if csv file has a header
    df = pd.read_csv(fpath, header = None, nrows = 1, on_bad_lines='warn')
    if df.loc[0,1] == 'LOG_ID':
        return pd.read_csv(filename, usecols=[1,3,18,20,21,22,23,24], skiprows=skip_rows, on_bad_lines='warn')
    else:
        return pd.read_csv(filename, names=col_names, usecols=[1,3,18,20,21,22,23,24], skiprows=skip_rows, on_bad_lines='warn')

In [ ]:
# saving measurment tables(csv) into parquet with extracting necessary columns
input_path = 'mover/EPIC_measurement'
for file in os.listdir(input_path)[1:]:
    # Continue when the file is not csv format
    if not 'csv' in file:
        continue
        
    fpath = f'{input_path}/{file}'
    print(f'{file} loading...', end='')
    
    df = read_csv(fpath, portion=1)
    print('saving...', end='')
    
    # Regex pattern for scientific notation (not hexadecimal)
    #sci_notation_regex = r'^-?\d+(\.\d+)?[eE][+\-]?\d+$'
    # Create a boolean mask for where the pattern matches
    #matches_mrn = df['MRN'].str.contains(sci_notation_regex, regex=True, na=False)
    #matches_log = df['logid'].str.contains(sci_notation_regex, regex=True, na=False)
    #print(f'MRN:{np.sum(matches_mrn)},logid:{np.sum(matches_log)}',end='...')
    
    #df['MRN'] = df['MRN'].apply(lambda x: int(x,16))
    #df['LOG_ID'] = df['LOG_ID'].apply(lambda x: int(x,16))
    
    df.to_parquet(path=f"{input_path}/parquet/{file.split('_')[0]}_measurement.parquet", engine='pyarrow', index=False)
    print(f'size {len(df)}rows...done')
    del df

3_2018-4633_FLOWSHEETS_20221014.csv loading...Sample 100 % of mover/EPIC_measurement/3_2018-4633_FLOWSHEETS_20221014.csv (8691538 lines)


/tmp/ipykernel_1216506/2352836814.py:25: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filename, usecols=[1,3,18,20,21,22,23,24], skiprows=skip_rows, on_bad_lines='warn')


saving...size 8691537rows...done
7_2018-4633_FLOWSHEETSB_part3.csv loading...Sample 100 % of mover/EPIC_measurement/7_2018-4633_FLOWSHEETSB_part3.csv (55673424 lines)
saving...size 55673424rows...done
2_2015-1924_flowsheets_201905221700.csv loading...Sample 100 % of mover/EPIC_measurement/2_2015-1924_flowsheets_201905221700.csv (46157623 lines)
saving...size 46157622rows...done
13_2018-4633_FLOWSHEETSB_part9.csv loading...Sample 100 % of mover/EPIC_measurement/13_2018-4633_FLOWSHEETSB_part9.csv (56797133 lines)
saving...size 56797133rows...done
10_2018-4633_FLOWSHEETSB_part6.csv loading...Sample 100 % of mover/EPIC_measurement/10_2018-4633_FLOWSHEETSB_part6.csv (56121661 lines)
saving...size 56121661rows...done
18_2018-4633_FLOWSHEETSB_part14.csv loading...Sample 100 % of mover/EPIC_measurement/18_2018-4633_FLOWSHEETSB_part14.csv (56605820 lines)
saving...size 56605820rows...done
6_2018-4633_FLOWSHEETSB_part2.csv loading...Sample 100 % of mover/EPIC_measurement/6_2018-4633_FLOWSHEETSB_

/tmp/ipykernel_1216506/2352836814.py:25: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filename, usecols=[1,3,18,20,21,22,23,24], skiprows=skip_rows, on_bad_lines='warn')


saving...size 487428148rows...done
9_2018-4633_FLOWSHEETSB_part5.csv loading...Sample 100 % of mover/EPIC_measurement/9_2018-4633_FLOWSHEETSB_part5.csv (55913122 lines)
saving...

In [702]:
input_path = 'mover/EPIC_measurement/parquet'

for file in os.listdir(input_path)[:]:
    fpath = f'{input_path}/{file}'
    print(f'{file} loading...', end='')
    
    df = pd.read_parquet(fpath)
    
    asdf

10_measurement.parquet loading...

NameError: name 'asdf' is not defined

In [703]:
df

,LOG_ID,MRN,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE,RECORD_TIME,MEAS_VALUE,UNITS
0,6d781634246e83f4,72aa50b6bc2691c0,Vital Signs,Temp source,PRE-OP,2021-10-16 08:00:00,Oral,None
1,223ba616545664e0,72aa50b6bc2691c0,Vital Signs,Temp source,POST-OP,2021-10-16 08:00:00,Oral,None
2,01379a9ef2074d85,72aa50b6bc2691c0,Vital Signs,Temp source,PRE-OP,2021-10-16 08:00:00,Oral,None
3,d374eb0e437fc602,72aa50b6bc2691c0,Vital Signs,Temp source,PRE-OP,2021-10-16 08:00:00,Oral,None
4,b241f36b21c60698,72aa50b6bc2691c0,Vital Signs,Temp source,PRE-OP,2021-10-16 08:00:00,Oral,None
...,...,...,...,...,...,...,...,...
56121656,285eae2a5ba03f69,d55c0558742e6803,Vital Signs,Arterial Line MAP (ART),POST-OP,2022-04-18 22:00:00,69,mmHg
56121657,92e85d09c206ff42,d55c0558742e6803,Vital Signs,Arterial Line MAP (ART),None,2022-04-18 22:00:00,69,mmHg
56121658,7e1765fcf298de75,d55c0558742e6803,Vital Signs,Arterial Line MAP (ART),None,2022-04-18 22:00:00,69,mmHg
56121659,38dc35cc5a61dd1f,d55c0558742e6803,Vital Signs,Arterial Line MAP (ART),POST-OP,2022-04-18 22:00:00,69,mmHg


In [11]:
# df = pd.read_csv('mover/EPIC_flowsheets/flowsheet_part1.csv') -> 487423896 rows × 8 columns
# df1 = pd.read_parquet('mover/EPIC_measurement/parquet/1_measurement.parquet') -> 487428148 rows × 8 columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487423896 entries, 0 to 487423895
Data columns (total 9 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Unnamed: 0        int64 
 1   LOG_ID            object
 2   MRN               object
 3   FLO_NAME          object
 4   FLO_DISPLAY_NAME  object
 5   RECORD_TYPE       object
 6   RECORDED_TIME     object
 7   MEAS_VALUE        object
 8   UNITS             object
dtypes: int64(1), object(8)
memory usage: 32.7+ GB


In [12]:
df1.isna().sum()

LOG_ID                      0
MRN                         0
FLO_NAME                    0
FLO_DISPLAY_NAME            0
RECORD_TYPE          43767230
RECORDED_TIME               0
MEAS_VALUE            1334139
UNITS               315402699
dtype: int64

In [10]:
df1.describe()

,LOG_ID,MRN,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE,RECORDED_TIME,MEAS_VALUE,UNITS
count,487428148,487428148,487428148,487428148,443660918,487428148,486094009,172025449
unique,23936,15698,91,113,3,1036202,98242,85
top,dfc05982f3b75ed3,e43f3b4bcb161d8a,Devices Testing Template,SpO2,POST-OP,2020-01-18 00:00:00,0,%
freq,895608,24181416,183574041,33058953,197224315,386381,25670126,67495315


### medication

In [555]:
df_medi = pd.read_csv(f'{input_path}/patient_medications.csv')

/tmp/ipykernel_1216506/4191568348.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_medi = pd.read_csv(f'{input_path}/patient_medications.csv')


In [217]:
df_medi = pd.read_csv(f'{input_path}/patient_medications.csv')
df = df_medi.iloc[:1000]

/tmp/ipykernel_1216506/4191568348.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_medi = pd.read_csv(f'{input_path}/patient_medications.csv')


In [343]:
# Sample DataFrame
df = pd.DataFrame({
    'medication': [
        'SODIUM CHLORIDE 0.9 % IV SOLN',
        'HYDROMORPHONE 10 MCG/ML BUPIVACAINE 0.1% PAIN MANAGEMENT PCEA (UCI)',
        'ROPIVACAINE HCL 5 MG/ML IJ SOLN',
        'INSULIN LISPRO (HUMAN) 100 UNIT/ML SC SOLN (UCI)',
        'BACITRACIN 50,000 UNITS IN 1000 ML NS IRRIGATION (OPTIME)',
        'DOXORUBICIN (ADRIAMYCIN) 50 MG, MITOMYCIN (MUTAMYCIN) 10 MG, IOHEXOL (OMNIPAQUE 240) 2040 MG IN STERILE WATER (PF) 10.5 ML TACE INJECTION (UCI)'
    ]
})

# Remove parentheses and their contents
df['cleaned'] = df['medication'].str.replace(r"\(.*?\)", "", regex=True)

# Remove numbers with their units and any following symbols (like %, ML, etc.)
df['cleaned'] = df['cleaned'].str.replace(r"\b\d+[\.,]?\d*.*?(?:\s*[A-Z]+/%|\s*[A-Z]+\b|\s*UNIT\S*)?", "", regex=True)

# Split the remaining string into words
df['ingredient'] = df['cleaned'].str.findall(r"\b[A-Za-z]+\b")

* Extract drug names (최종본)

In [615]:
df = df_medi[['DISPLAY_NAME', 'MEDICATION_NM']].drop_duplicates(subset=['MEDICATION_NM'])
#drugs = df_medi[['DISPLAY_NAME', 'MEDICATION_NM', 'ingredient']].drop_duplicates(subset=['MEDICATION_NM', 'ingredient'])

# Remove parentheses and their contents
df['cleaned'] = df['MEDICATION_NM'].str.lower().str.replace(r"\(.*?\)", "", regex=True)

# Step 1: Use regular expressions to match the patterns and extract the relevant parts
# The regex pattern will look for drug names, potentially followed by dosages in parentheses or next to units
#pattern = r'(\b[a-z]+(?:\s[a-z]+)*(?:\s?\([a-z]+\))?)\s*\d*\s*%?\s*[a-z]+\/?[a-z]*'
#pattern = r'(\b[a-z]+(?:-[a-z]+)*\b)(?![a-z])'
pattern = r'(\b[a-zA-Z]+(?:-[a-zA-Z]+)*(?:\s[a-zA-Z]+(?:-[a-zA-Z]+)*)*)(?:\s\([^)]+\))?\s*(?:\d+(?:\.\d+)?\s*(?:%|MG|ML|GM|G|MC|ME)?(?:/[A-Z]+)?)?'

# Extractall will return a DataFrame with a match for each capture group per row
matches = df['cleaned'].str.extractall(pattern)
matches = matches[matches[0].str.len() > 2]

# Unstack the matches, which puts each match in its own column, and then join those columns
matches = matches.unstack().apply(lambda row: row.dropna().tolist(), axis=1)
df['ingredient'] = matches

df = df.explode('ingredient').reset_index(drop=True)
drop_vals = ['or tab', 'ads overrid', 'iv sol', 'iv soln', 'mg po tabs', 'mg ij solr', 'ij sol', 'ij soln', 'ml ij soln', 
             'mcg', 'iv emul', 'iv sosy', 'unt', 'ri soln', 'soln optime', 'gm ij solr', 'gm in swfi', 'swfi', 'ex swab',
             'mg or tab', 'mcg or tabs', 'or cap', 'po tab', 'sol', 'op sol', 'or sus', 'po cap', 'or sol', 'ml infusion', 
             'mg or tabs', 'mg po tab', 'mg re supp', 'or soln', 'mg or tbec', 'ml sc soln', 'or pack', 'ads override', 
             'mg or chew', 'meq', 'mg in', 'units in', 'ex gel', 'meq or tbcr', 'or t', 'or c', 'po t', 'sc sol', 'sc soln']
df = df[~df['ingredient'].isin(drop_vals)].dropna()

# Remove the ingredients that contain vitamin
df = df[~df['ingredient'].str.contains('vitamin')]

df = df.assign(ingredient=df['ingredient'].str.split('-')).explode('ingredient').reset_index(drop=True)

In [616]:
df

,DISPLAY_NAME,MEDICATION_NM,cleaned,ingredient
0,levothyroxine (SYNTHROID) tablet 125 mcg,LEVOTHYROXINE SODIUM 125 MCG OR TABS,levothyroxine sodium 125 mcg or tabs,levothyroxine sodium
1,atorvastatin (LIPITOR) tablet 40 mg,ATORVASTATIN CALCIUM 40 MG OR TABS,atorvastatin calcium 40 mg or tabs,atorvastatin calcium
2,dextrose-sodium chloride 5%-0.9% infusion,DEXTROSE-NACL 5-0.9 % IV SOLN,dextrose-nacl 5-0.9 % iv soln,dextrose
3,dextrose-sodium chloride 5%-0.9% infusion,DEXTROSE-NACL 5-0.9 % IV SOLN,dextrose-nacl 5-0.9 % iv soln,nacl
4,sodium chloride 0.9 % flush 50 mL,SODIUM CHLORIDE 0.9 % IJ SOLN,sodium chloride 0.9 % ij soln,sodium chloride
...,...,...,...,...
7443,Olmesartan-amLODIPine-HCTZ 40-5-12.5 MG TABS 1...,OLMESARTAN-AMLODIPINE-HCTZ 40-5-12.5 MG PO TABS,olmesartan-amlodipine-hctz 40-5-12.5 mg po tabs,olmesartan
7444,Olmesartan-amLODIPine-HCTZ 40-5-12.5 MG TABS 1...,OLMESARTAN-AMLODIPINE-HCTZ 40-5-12.5 MG PO TABS,olmesartan-amlodipine-hctz 40-5-12.5 mg po tabs,amlodipine
7445,Olmesartan-amLODIPine-HCTZ 40-5-12.5 MG TABS 1...,OLMESARTAN-AMLODIPINE-HCTZ 40-5-12.5 MG PO TABS,olmesartan-amlodipine-hctz 40-5-12.5 mg po tabs,hctz
7446,DOXOrubicin 50 mg in sterile water (PF) 2 mL T...,TACE - DOXORUBICIN 50 MG (OPTIME),tace - doxorubicin 50 mg,tace


In [617]:
### 문제점 발견: potassium phosphates 여서 potassium phosphate로 매핑 안됨.

df['ingredients'] = df['ingredient'].str.split()
df['ingredient3'] = df['ingredients'].apply(lambda x: ' '.join(x[:3])).str.strip()
df['ingredient2'] = df['ingredients'].apply(lambda x: ' '.join(x[:2])).str.strip()
df['ingredient1'] = df['ingredients'].apply(lambda x: ' '.join(x[:1])).str.strip()

# Map the source concept of drugs into standard concept (RxNorm)
rxnorm_concepts = df_concept[((df_concept['vocabulary_id'] == 'RxNorm') | (df_concept['vocabulary_id'] == 'RxNorm Extension')) & (df_concept['standard_concept'] == 'S')][['concept_name', 'concept_id']]
rxnorm_concepts['concept_name'] = rxnorm_concepts['concept_name'].str.lower()

df['concept_id3'] = df.merge(rxnorm_concepts, left_on='ingredient3', right_on='concept_name', how='left')['concept_id']
df['concept_id2'] = df.merge(rxnorm_concepts, left_on='ingredient2', right_on='concept_name', how='left')['concept_id']
df['concept_id1'] = df.merge(rxnorm_concepts, left_on='ingredient1', right_on='concept_name', how='left')['concept_id']

df['concept_id'] = df[['concept_id3', 'concept_id2', 'concept_id1']].bfill(axis=1).iloc[:, 0]
df['concept_id'] = df['concept_id'].astype('Int64')

In [618]:
df['concept_id'].isna().sum()

3411

In [619]:
#df.dropna(subset='concept_id', inplace=True)
df = df[['MEDICATION_NM', 'ingredient', 'concept_id']]
df.to_csv('results/labeled/mover_medi_mapped.csv', index=False)

In [620]:
df

,MEDICATION_NM,ingredient,concept_id
0,LEVOTHYROXINE SODIUM 125 MCG OR TABS,levothyroxine sodium,1501700
1,ATORVASTATIN CALCIUM 40 MG OR TABS,atorvastatin calcium,1545958
2,DEXTROSE-NACL 5-0.9 % IV SOLN,dextrose,<NA>
3,DEXTROSE-NACL 5-0.9 % IV SOLN,nacl,<NA>
4,SODIUM CHLORIDE 0.9 % IJ SOLN,sodium chloride,967823
...,...,...,...
7443,OLMESARTAN-AMLODIPINE-HCTZ 40-5-12.5 MG PO TABS,olmesartan,40226742
7444,OLMESARTAN-AMLODIPINE-HCTZ 40-5-12.5 MG PO TABS,amlodipine,1332418
7445,OLMESARTAN-AMLODIPINE-HCTZ 40-5-12.5 MG PO TABS,hctz,<NA>
7446,TACE - DOXORUBICIN 50 MG (OPTIME),tace,<NA>


* drug counts

In [337]:
import time

# previous algorithm for drug name extraction
start = time.time()
df_medi = df_medi.assign(ingredient=df_medi['MEDICATION_NM'].str.split(',')).explode('ingredient').reset_index(drop=True)
print(time.time() - start)
start = time.time()

df_medi['ingredient'] = df_medi['ingredient'].str.extract(r'([^0-9(%]+)')[0].str.strip().str.lower()
print(time.time() - start)
start = time.time()

df_medi = df_medi.assign(ingredient=df_medi['ingredient'].str.split('-')).explode('ingredient').reset_index(drop=True)
print(time.time() - start)

KeyError: 'medication'

In [331]:
# 굳이 df_medi에서 돌릴 필요 없고, drugs만 이용
drugs = df_medi[['DISPLAY_NAME', 'MEDICATION_NM', 'ingredient']].drop_duplicates(subset=['MEDICATION_NM', 'ingredient'])
drugs[drugs['ingredient'].str.contains('vitamin')].to_csv('vitamins.csv', index=False)

In [326]:
# Medi_counts
drugs_per_subject = df_medi.drop_duplicates(subset=['MRN', 'ingredient'])
drug_counts = drugs_per_subject['ingredient'].value_counts().to_frame()
drug_counts.reset_index(inplace=True)
# 문제는 ingredient 하나에 여러개 medication이 대응될 수 있음 (medication을 split해서 ingredient를 만들어서, 원래 ingredient 하나 자체가 medication인 경우와 중복)
drug_counts['medication'] = drug_counts.merge(drugs_per_subject.drop_duplicates(['ingredient']), on='ingredient', how='left')['MEDICATION_NM']

# Map the source concept of drugs into standard concept (RxNorm)
rxnorm_concepts = df_concept[((df_concept['vocabulary_id'] == 'RxNorm') | (df_concept['vocabulary_id'] == 'RxNorm Extension')) & (df_concept['standard_concept'] == 'S')][['concept_name', 'concept_id']]
rxnorm_concepts['concept_name'] = rxnorm_concepts['concept_name'].str.lower()

drug_mapped = drug_counts.merge(rxnorm_concepts, left_on='ingredient', right_on='concept_name', how='left')

drug_mismatch = drug_mapped[drug_mapped['concept_id'].isna()]

drug_counts['perc'] = drug_counts['count'] / len(df_medi['MRN'].unique()) * 100
#drug_counts['mismatch'] = np.where(drug_counts['ingredient'].isin(drug_mismatch['ingredient']), 1, np.nan)
#drug_counts.to_csv('results/mover_medi_counts.csv', index=False)

# Remove rows that have empty ingredient
drug_counts = drug_counts[drug_counts['ingredient']!='']
drug_counts

,ingredient,count,medication,perc
0,propofol,36910,PROPOFOL 1000 MG/100 ML INFUSION (UCI),95.391931
1,fentanyl citrate,35961,FENTANYL CITRATE (PF) 100 MCG/2ML IJ SOLN,92.939291
2,acetaminophen,34695,HYDROCODONE-ACETAMINOPHEN 7.5-325 MG/15ML OR SOLN,89.667382
3,ondansetron hcl,34613,ONDANSETRON HCL 4 MG/2ML IV SOLN,89.455457
4,plasma,33194,PLASMA-LYTE A IV BOLUS (UCI),85.788127
...,...,...,...,...
2084,lutein po tabs,1,OCUVITE-LUTEIN PO TABS,0.002584
2085,ocuvite,1,OCUVITE-LUTEIN PO TABS,0.002584
2086,clorazepate dipotassium,1,CLORAZEPATE DIPOTASSIUM 7.5 MG OR TABS,0.002584
2087,feeding tubes misc,1,FEEDING TUBES MISC,0.002584


In [258]:
# Add an auxiliary column to maintain the order
drug_counts['order'] = range(len(drug_counts))

# Step 1: Extract the first word of 'ingredient' if it has more than one word
drug_counts['first_word'] = drug_counts['ingredient'].str.split().str[0]

# Step 2: Merge with rxnorm_concepts on 'ingredient' for exact matches
exact_matches = pd.merge(drug_counts, rxnorm_concepts, left_on='ingredient', right_on='concept_name', how='left')

# Step 3: Identify which entries didn't get a match
non_matches = drug_counts[~drug_counts['ingredient'].isin(exact_matches['concept_name'])]
exact_matches.dropna(subset='concept_id', inplace=True)

# Step 4: Merge the non-matches with the 'concept_name' based on 'first_word'
partial_matches = pd.merge(non_matches, rxnorm_concepts, left_on='first_word', right_on='concept_name', how='left')

# Step 5: Combine the exact matches with the partial matches
combined_matches = pd.concat([exact_matches, partial_matches]).sort_values(by='order')

# Step 6: Drop the temporary 'first_word' columns and any duplicates that might have arisen
final_df = combined_matches.drop_duplicates(['first_word', 'order']).drop(columns=['first_word', 'order'])

# Check mismatch
final_df['mismatch'] = (final_df['concept_id'].isna()).astype(int)

final_df.to_csv('results/mover_medi_counts.csv', index=False)

In [260]:
final_df

,ingredient,count,medication,perc,concept_name,concept_id,mismatch
0,propofol,36910,PROPOFOL 1000 MG/100 ML INFUSION (UCI),95.391931,propofol,753626.0,0
0,fentanyl citrate,35961,FENTANYL CITRATE (PF) 100 MCG/2ML IJ SOLN,92.939291,fentanyl,1154029.0,0
2,acetaminophen,34695,HYDROCODONE-ACETAMINOPHEN 7.5-325 MG/15ML OR SOLN,89.667382,acetaminophen,1125315.0,0
1,ondansetron hcl,34613,ONDANSETRON HCL 4 MG/2ML IV SOLN,89.455457,ondansetron,1000560.0,0
2,plasma,33194,PLASMA-LYTE A IV BOLUS (UCI),85.788127,NaN,NaN,1
...,...,...,...,...,...,...,...
1406,lutein po tabs,1,OCUVITE-LUTEIN PO TABS,0.002584,lutein,1310353.0,0
1407,ocuvite,1,OCUVITE-LUTEIN PO TABS,0.002584,NaN,NaN,1
1408,clorazepate dipotassium,1,CLORAZEPATE DIPOTASSIUM 7.5 MG OR TABS,0.002584,clorazepate,790253.0,0
1409,feeding tubes misc,1,FEEDING TUBES MISC,0.002584,NaN,NaN,1


In [312]:
vitamins = df_medi[df_medi['MEDICATION_NM'].str.contains('VITAMIN')]
pd.DataFrame({'medication': vitamins['MEDICATION_NM'].unique()}).to_csv('vitamins.csv')
vitamins

,ENC_TYPE_C,ENC_TYPE_NM,LOG_ID,MRN,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,END_DATE,ORDER_STATUS_NM,RECORD_TYPE,MAR_ACTION_NM,MED_ACTION_TIME,ADMIN_SIG,DOSE_UNIT_NM,MED_ROUTE_NM,ingredient
1255,3,Hospital Encounter,0c60fe1be8d81abb,c86284827c4abd04,2020-12-22 00:00:00,Inpatient,8654,"vitamin B-12 (CYANOCOBALAMIN) tablet TABS 1,00...",VITAMIN B-12 1000 MCG OR TABS,2020-12-23 00:00:00,2020-12-31 00:00:00,Discontinued,NaN,Given,2020-12-23 08:38:00,1000.0,mcg,Oral,vitamin b
1256,3,Hospital Encounter,0c60fe1be8d81abb,c86284827c4abd04,2020-12-22 00:00:00,Inpatient,8654,"vitamin B-12 (CYANOCOBALAMIN) tablet TABS 1,00...",VITAMIN B-12 1000 MCG OR TABS,2020-12-23 00:00:00,2020-12-31 00:00:00,Discontinued,NaN,Given,2020-12-23 08:38:00,1000.0,mcg,Oral,
1257,3,Hospital Encounter,0c60fe1be8d81abb,c86284827c4abd04,2020-12-22 00:00:00,Inpatient,8654,"vitamin B-12 (CYANOCOBALAMIN) tablet TABS 1,00...",VITAMIN B-12 1000 MCG OR TABS,2020-12-23 00:00:00,2020-12-31 00:00:00,Discontinued,NaN,Missed,2020-12-25 09:00:00,1000.0,mcg,Oral,vitamin b
1258,3,Hospital Encounter,0c60fe1be8d81abb,c86284827c4abd04,2020-12-22 00:00:00,Inpatient,8654,"vitamin B-12 (CYANOCOBALAMIN) tablet TABS 1,00...",VITAMIN B-12 1000 MCG OR TABS,2020-12-23 00:00:00,2020-12-31 00:00:00,Discontinued,NaN,Missed,2020-12-25 09:00:00,1000.0,mcg,Oral,
1259,3,Hospital Encounter,0c60fe1be8d81abb,c86284827c4abd04,2020-12-22 00:00:00,Inpatient,8654,"vitamin B-12 (CYANOCOBALAMIN) tablet TABS 1,00...",VITAMIN B-12 1000 MCG OR TABS,2020-12-23 00:00:00,2020-12-31 00:00:00,Discontinued,NaN,MAR Hold,2020-12-28 05:52:22,NaN,mcg,Oral,vitamin b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30994328,3,Hospital Encounter,cca6056a5fdd5d65,0502759842cdf93c,2021-05-08 00:00:00,Inpatient,13108,vitamin D3 (VITAMIN D) tablet 400 Units,VITAMIN D3 400 UNIT OR TABS,2021-05-09 00:00:00,2021-05-27 00:00:00,Discontinued,POST-OP,Given,2021-05-15 08:56:00,400.0,Units,NG Tube,vitamin d
30994329,3,Hospital Encounter,cca6056a5fdd5d65,0502759842cdf93c,2021-05-08 00:00:00,Inpatient,13108,vitamin D3 (VITAMIN D) tablet 400 Units,VITAMIN D3 400 UNIT OR TABS,2021-05-09 00:00:00,2021-05-27 00:00:00,Discontinued,POST-OP,Given,2021-05-18 10:00:00,400.0,Units,NG Tube,vitamin d
30994330,3,Hospital Encounter,cca6056a5fdd5d65,0502759842cdf93c,2021-05-08 00:00:00,Inpatient,13108,vitamin D3 (VITAMIN D) tablet 400 Units,VITAMIN D3 400 UNIT OR TABS,2021-05-09 00:00:00,2021-05-27 00:00:00,Discontinued,POST-OP,Given,2021-05-21 10:00:00,400.0,Units,NG Tube,vitamin d
30994331,3,Hospital Encounter,cca6056a5fdd5d65,0502759842cdf93c,2021-05-08 00:00:00,Inpatient,13108,vitamin D3 (VITAMIN D) tablet 400 Units,VITAMIN D3 400 UNIT OR TABS,2021-05-09 00:00:00,2021-05-27 00:00:00,Discontinued,POST-OP,Given,2021-05-24 09:31:00,400.0,Units,NG Tube,vitamin d


In [327]:
ing_vit = drug_counts[drug_counts['medication'].str.contains('VITAMIN')]
ing_vit.to_csv('vitamins.csv')
ing_vit

,ingredient,count,medication,perc
121,vitamin d,2593,VITAMIN D3 1000 UNIT OR TABS,6.701471
173,vitamin c,1428,VITAMIN C 500 MG OR TABS,3.690590
191,calcium carbonate,1158,CALCIUM CARBONATE-VITAMIN D 250-125 MG-UNIT OR...,2.992789
221,vitamin b,954,VITAMIN B-12 1000 MCG OR TABS,2.465562
327,ascorbic acid,502,ASCORBIC ACID (VITAMIN C) IVPB (UCI),1.297392
450,fluids with multivitamins infusion,237,FLUIDS WITH MULTIVITAMINS INFUSION (UCI),0.612514
529,vitamin a,154,VITAMIN A 10000 UNIT OR CAPS (UCI),0.398005
637,vitamin e,87,VITAMIN E 400 UNIT OR CAPS,0.224847
664,vitamins a & d ex oint,78,VITAMINS A & D EX OINT,0.201587
977,poly,17,POLY-VITAMINS OR SOLN,0.043936


# PERSON

In [19]:
import os

save_path = 'MOVER_ETL'
if not os.path.exists(save_path):
    os.mkdir(save_path)
    
start_index = {
    'person': 1000000,
    'observation_period': 2000000,
    'visit_occurrence': 3000000,
    'visit_detail': 4000000,
    'condition_occurrence': 5000000,
    'drug_exposure': 6000000,
    'procedure_occurrence': 7000000,
    'measurement': 8000000,
    'death': 9000000,
    'note': 10000000,
    'location': 20000000 
    }

In [733]:
# Create an empty dataframe with defined columns for PERSON table
df_person = pd.DataFrame()

# Assign unique IDs to each distinct 'MRN' from the operations data
unique_ids = df_info['MRN'].unique()
df_person['PERSON_ID'] = start_index['person'] + np.arange(1, len(unique_ids) + 1)
df_person['MRN'] = unique_ids

# Merge relevant columns from the operations dataframe with the PERSON dataframe based on 'MRN'
usecols = ['MRN', 'BIRTH_DATE', 'SURGERY_DATE', 'SEX']
df_person = df_person.merge(df_info[usecols], on = 'MRN', how='left')
# Ensure only the latest discharge_time is retained for each unique PERSON_ID
df_person.drop_duplicates(subset = 'PERSON_ID', keep = 'last', inplace = True, ignore_index = True)

# Map gender values ('M' or 'F') to corresponding GENDER_CONCEPT_ID values
df_person['GENDER_CONCEPT_ID'] = df_person['SEX'].map({'Male': 8507, 'Female': 8532})

# Remove any rows with missing gender values
df_person.dropna(subset=['GENDER_CONCEPT_ID'])

# Calculate and assign the year of birth based on age and the start date
df_person['YEAR_OF_BIRTH'] = df_person['SURGERY_DATE'].dt.year - df_person['BIRTH_DATE']
# Compute the exact birth datetime using age and start date
df_person['BIRTH_DATETIME'] = df_person['YEAR_OF_BIRTH'].apply(lambda x: datetime(year=x, month=1, day=1))

# Set RACE_CONCEPT_ID to indicate all individuals are
#df_person['RACE_CONCEPT_ID'] = 

# Assign default values for LOCATION_ID and PROVIDER_ID
df_person['LOCATION_ID'] = 'mover'
#df_person['PROVIDER_ID'] = 0

# Populate source value columns based on values from the operations data
df_person['PERSON_SOURCE_VALUE'] = df_person['MRN']
df_person['GENDER_SOURCE_VALUE'] = df_person['SEX']
#df_person['RACE_SOURCE_VALUE'] = df_person['race']
#df_person['RACE_SOURCE_CONCEPT_ID'] = 8515

# Remove columns that aren't part of the final PERSON table format
df_person.drop(columns=usecols, inplace=True)

# Write the processed data to a CSV file
df_person.to_csv(f'{save_path}/MOVER_PERSON.csv', index=False)

In [734]:
df_person

,PERSON_ID,GENDER_CONCEPT_ID,YEAR_OF_BIRTH,BIRTH_DATETIME,LOCATION_ID,PERSON_SOURCE_VALUE,GENDER_SOURCE_VALUE
0,1000001,8532.0,1972,1972-01-01,mover,2e92522a5f2a2a22,Female
1,1000002,8507.0,1938,1938-01-01,mover,b7a91e623d957d8f,Male
2,1000003,8532.0,1970,1970-01-01,mover,b68d6c7198f30f73,Female
3,1000004,8532.0,1964,1964-01-01,mover,46aa48e89b9c50e8,Female
4,1000005,8532.0,1975,1975-01-01,mover,c2756216cc89b795,Female
...,...,...,...,...,...,...,...
39680,1039681,8532.0,1998,1998-01-01,mover,6abf6ccb5b2fc64a,Female
39681,1039682,8532.0,1964,1964-01-01,mover,902b15f38d15fc65,Female
39682,1039683,8532.0,1983,1983-01-01,mover,537be76fbfaec3d9,Female
39683,1039684,8507.0,1956,1956-01-01,mover,aecf54e7e0077c0b,Male


# OBSERVATION_PERIOD

* Definition of OBSERVATION_PERIOD record : 각 subject_id 마다 첫 입원 시점부터 operations.csv, vitals.csv, ward_vitals.csv, labs.csv, medications.csv 에 기록된 가장 마지막 시점까지를 observation-period record로 정의 (즉, 한 사람 당 하나의 record)

    * 다른 가능한 정의) 하나의 subject_id에 대해 각 hadm_id를 하나의 Clinical Event로 정의함. 이 Clinical Event의 시작 시점과 끝 시점은 operations.csv

In [737]:
# Create an empty dataframe for OBSERVATION_PERIOD table
df_obs = pd.DataFrame()

# Copy PERSON_ID from PERSON table to OBSERVATION_PERIOD_ID and PERSON_ID columns in OBSERVATION_PERIOD table
df_obs['OBSERVATION_PERIOD_ID'] = start_index['observation_period'] - start_index['person'] + df_person['PERSON_ID']
df_obs['PERSON_ID'] = df_person['PERSON_ID']
# Copy PERSON_SOURCE_VALUE from PERSON table to MRN in OBSERVATION_PERIOD table for merging purposes
df_obs['MRN'] = df_person['PERSON_SOURCE_VALUE']


usecols = ['MRN', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME']
# Get the earliest admission time and latest discharge time for each MRN
grouped = df_info.groupby('MRN').agg({
    'HOSP_ADMSN_TIME': 'min',
    'HOSP_DISCH_TIME': 'max'
}).reset_index()
# Merge to observation table
df_obs = df_obs.merge(grouped, on='MRN', how='left')

# Set the OBSERVATION_PERIOD_START_DATE to admission time
df_obs['OBSERVATION_PERIOD_START_DATE'] = df_obs['HOSP_ADMSN_TIME'].dt.date
# Set the OBSERVATION_PERIOD_END_DATE to discharge time
df_obs['OBSERVATION_PERIOD_END_DATE'] = df_obs['HOSP_DISCH_TIME'].dt.date

# Assign the PERIOD_TYPE_CONCEPT_ID indicating the data source is an EHR
df_obs['PERIOD_TYPE_CONCEPT_ID'] = 32817

# Remove columns that aren't part of the final OBSERVATION_PERIOD table format
df_obs.drop(columns=usecols, inplace=True)

# Write the processed data to a CSV file
df_obs.to_csv(f'{save_path}/MOVER_OBSERVATION_PERIOD.csv', index=False)

In [738]:
df_obs

,OBSERVATION_PERIOD_ID,PERSON_ID,OBSERVATION_PERIOD_START_DATE,OBSERVATION_PERIOD_END_DATE,PERIOD_TYPE_CONCEPT_ID
0,2000001,1000001,2018-12-20,2021-02-06,32817
1,2000002,1000002,2018-07-26,2019-10-18,32817
2,2000003,1000003,2019-04-08,2019-04-11,32817
3,2000004,1000004,2018-12-03,2018-12-04,32817
4,2000005,1000005,2019-04-19,2019-04-20,32817
...,...,...,...,...,...
39680,2039681,1039681,2021-11-15,2021-11-15,32817
39681,2039682,1039682,2021-08-15,2021-08-15,32817
39682,2039683,1039683,2022-12-01,2022-12-02,32817
39683,2039684,1039684,2021-06-07,2021-08-03,32817


# VISIT_OCCURRENCE

* hadm_id 단위로 visit으로 정의

In [26]:
# Create an empty dataframe for VISIT_OCCURRENCE table
df_visit_occ = pd.DataFrame(columns=['VISIT_OCCURRENCE_ID'])


# Copy PERSON_ID values from df_person to df_visit_occ
df_visit_occ['PERSON_ID'] = df_person['PERSON_ID']

# Copy PERSON_SOURCE_VALUE values (as MRN) from df_person to df_visit_occ
df_visit_occ['MRN'] = df_person['PERSON_SOURCE_VALUE']

# Merge visit_occurrence data with operation data based on 'MRN'
usecols = ['LOG_ID', 'MRN', 'DISCH_DISP', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME', 'PATIENT_CLASS_GROUP']
df_visit_occ = df_visit_occ.merge(df_info[usecols], on = 'MRN', how = 'left')

# Assign sequential IDs starting from 1 to VISIT_OCCURRENCE_ID column
df_visit_occ['VISIT_OCCURRENCE_ID'] = start_index['visit_occurrence'] + np.arange(1, len(df_visit_occ) + 1)

# Assign 9201 for Inpatient Visit and 9202 for Outpatient Visit.
df_visit_occ['VISIT_CONCEPT_ID'] = df_visit_occ['PATIENT_CLASS_GROUP'].map({'Inpatient':9201, 'Outpatient':9202})
df_visit_occ['VISIT_SOURCE_VALUE'] = df_visit_occ['PATIENT_CLASS_GROUP']

# Assign VISIT_START_DATETIME based on hospital admission time
df_visit_occ['VISIT_START_DATETIME'] = pd.to_datetime(df_visit_occ['HOSP_ADMSN_TIME'])
# Extract the date part for VISIT_START_DATE
df_visit_occ['VISIT_START_DATE'] = df_visit_occ['VISIT_START_DATETIME'].dt.date

# Assign VISIT_END_DATETIME based on hospital discharge time
df_visit_occ['VISIT_END_DATETIME'] = pd.to_datetime(df_visit_occ['HOSP_DISCH_TIME'])
# Extract the date part for VISIT_END_DATE
df_visit_occ['VISIT_END_DATE'] = df_visit_occ['VISIT_END_DATETIME'].dt.date

# Assign the VISIT_TYPE_CONCEPT_ID indicating the data source is an EHR
df_visit_occ['VISIT_TYPE_CONCEPT_ID'] = 32817

df_visit_occ['DISCHARGED_TO_SOURCE_VALUE'] = df_visit_occ['DISCH_DISP']

## Mapping PRECEDING_VISIT_OCCURRENCE_ID
# Generate a column with the previous 'MRN' for determining preceding visit occurrence
df_visit_occ['prev_MRN'] = df_visit_occ['MRN'].shift(1)
# Create a new boolean column 'nadm' to check if the current row's MRN matches the previous one
df_visit_occ['nadm'] = df_visit_occ['MRN'] == df_visit_occ['prev_MRN']
# Set the first row's 'nadm' value to False since there's no preceding record
df_visit_occ.at[0, 'nadm'] = False 
# Compute PRECEDING_VISIT_OCCURRENCE_ID based on 'nadm'
df_visit_occ['PRECEDING_VISIT_OCCURRENCE_ID'] = np.where(df_visit_occ['nadm'], df_visit_occ['VISIT_OCCURRENCE_ID'].shift(1), np.nan)
df_visit_occ['PRECEDING_VISIT_OCCURRENCE_ID'] = df_visit_occ['PRECEDING_VISIT_OCCURRENCE_ID'].astype('Int64')
    
# Assign default values for PROVIDER_ID and CARE_SITE_ID
#df_visit_occ['PROVIDER_ID'] = 0
#df_visit_occ['CARE_SITE_ID'] = 0

 # Remove columns that aren't part of the final VISIT_OCCURRENCE table format
df_visit_occ.drop(columns=usecols[1:]+['prev_MRN', 'nadm'], inplace=True)

# Save the processed data to a CSV file
df_visit_occ.to_csv(f'{save_path}/MOVER_VISIT_OCCURRENCE.csv', index=False)
df_visit_occ.to_parquet(f'{save_path}/MOVER_VISIT_OCCURRENCE.parquet')

In [788]:
df_visit_occ

,VISIT_OCCURRENCE_ID,PERSON_ID,LOG_ID,VISIT_CONCEPT_ID,VISIT_SOURCE_VALUE,VISIT_START_DATETIME,VISIT_START_DATE,VISIT_END_DATETIME,VISIT_END_DATE,VISIT_TYPE_CONCEPT_ID,DISCHARGED_TO_SOURCE_VALUE,PRECEDING_VISIT_OCCURRENCE_ID
0,3000001,1000001,cd1636c6279d73a0,9202,Outpatient,2018-12-20 11:20:00,2018-12-20,2018-12-21 12:15:00,2018-12-21,32817,Home Routine,<NA>
1,3000002,1000001,f8378adc3308508b,9201,Inpatient,2021-02-03 18:51:00,2021-02-03,2021-02-06 20:00:00,2021-02-06,32817,Home Routine,3000001
2,3000003,1000002,fc53c06ee2cfe438,9202,Outpatient,2019-09-27 09:02:00,2019-09-27,2019-10-18 18:13:00,2019-10-18,32817,Hospice Facility,<NA>
3,3000004,1000002,542843d28ab06693,9201,Inpatient,2019-09-27 09:02:00,2019-09-27,2019-10-18 18:13:00,2019-10-18,32817,Hospice Facility,3000003
4,3000005,1000002,5b494bfd62a3afac,9201,Inpatient,2019-09-27 09:02:00,2019-09-27,2019-10-18 18:13:00,2019-10-18,32817,Hospice Facility,3000004
...,...,...,...,...,...,...,...,...,...,...,...,...
65723,3065724,1039681,b50eb5fd9e58de7c,9202,Outpatient,2021-11-15 12:44:00,2021-11-15,2021-11-15 18:27:00,2021-11-15,32817,Home Routine,<NA>
65724,3065725,1039682,1b41ee229f341d06,9202,Outpatient,2021-08-15 05:32:00,2021-08-15,2021-08-15 11:35:00,2021-08-15,32817,Home Routine,<NA>
65725,3065726,1039683,ab4064a10f4d1cc9,9201,Inpatient,2022-12-01 05:04:00,2022-12-01,2022-12-02 10:44:00,2022-12-02,32817,Home Routine,<NA>
65726,3065727,1039684,b619db184a3f7d86,9201,Inpatient,2021-06-07 03:35:00,2021-06-07,2021-08-03 13:00:00,2021-08-03,32817,Skilled Nursing Facility,<NA>


# VISIT_DETAIL

* **Table Description**  
  The VISIT_DETAIL table is an optional table used to represents details of each record in the parent VISIT_OCCURRENCE table. A good example of this would be the movement between units in a hospital during an inpatient stay or claim lines associated with a one insurance claim. For every record in the VISIT_OCCURRENCE table there may be 0 or more records in the VISIT_DETAIL table with a 1:n relationship where n may be 0. The VISIT_DETAIL table is structurally very similar to VISIT_OCCURRENCE table and belongs to the visit domain.


  VISIT_DETAIL: ICU 기록이 있는 경우 매핑

In [27]:
# Create a new DataFrame for VISIT_DETAIL data with the specified columns
df_visit_detail = pd.DataFrame(columns=['VISIT_DETAIL_ID'])

# Populate the PERSON_ID and MRN columns with data from the df_person DataFrame
df_visit_detail['PERSON_ID'] = df_person['PERSON_ID']
df_visit_detail['MRN'] = df_person['PERSON_SOURCE_VALUE']

# Integrate visit detail data with operational data from df_op using 'MRN'
usecols = ['LOG_ID', 'MRN', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME', 'ICU_ADMIN']
df_visit_detail = df_visit_detail.merge(df_info[usecols], on='MRN')

# Remove rows that do not have ICU_ADMIN record
df_visit_detail = df_visit_detail[df_visit_detail['ICU_ADMIN']=='Yes'].reset_index(drop=True)

# Generate unique sequential IDs for VISIT_DETAIL_ID
df_visit_detail['VISIT_DETAIL_ID'] = start_index['visit_detail'] + np.arange(len(df_visit_detail)) + 1

# Designate a concept ID representing ICU visits
df_visit_detail['VISIT_DETAIL_CONCEPT_ID'] = 32037

# Assign visit start and end datetime values to hosptial admission, discharge times
df_visit_detail['VISIT_DETAIL_START_DATETIME'] = df_visit_detail['HOSP_ADMSN_TIME']
df_visit_detail['VISIT_DETAIL_START_DATE'] = df_visit_detail['VISIT_DETAIL_START_DATETIME'].dt.date
df_visit_detail['VISIT_DETAIL_END_DATETIME'] = df_visit_detail['HOSP_DISCH_TIME']
df_visit_detail['VISIT_DETAIL_END_DATE'] = df_visit_detail['VISIT_DETAIL_END_DATETIME'].dt.date

# Specify the concept ID for the visit detail type as sourced from EHR
df_visit_detail['VISIT_DETAIL_TYPE_CONCEPT_ID'] = 32817

# Determine preceding visits for each entry by comparing 'LOG_ID' with its previous entry
df_visit_detail['prev_LOG_ID'] = df_visit_detail['LOG_ID'].shift(1)
df_visit_detail['nadm'] = df_visit_detail['LOG_ID'] == df_visit_detail['prev_LOG_ID']
df_visit_detail.at[0, 'nadm'] = False  # The first entry won't have a preceding visit
df_visit_detail['PRECEDING_VISIT_DETAIL_ID'] = np.where(df_visit_detail['nadm'], df_visit_detail['VISIT_DETAIL_ID'].shift(1), np.nan)
df_visit_detail['PRECEDING_VISIT_DETAIL_ID'] = df_visit_detail['PRECEDING_VISIT_DETAIL_ID'].astype('Int64')

# Merge with df_visit_occ to fetch 'VISIT_OCCURRENCE_ID' values
df_visit_detail['VISIT_OCCURRENCE_ID'] = df_visit_detail.merge(df_visit_occ[['LOG_ID', 'VISIT_OCCURRENCE_ID']], on='LOG_ID', how='left')['VISIT_OCCURRENCE_ID']

# Remove columns that aren't part of the final VISIT_DETAIL table format
df_visit_detail.drop(columns=usecols[1:]+['prev_LOG_ID', 'nadm'], inplace=True)
           
# Save the processed VISIT_DETAIL data to a CSV file
df_visit_detail.to_csv('INSPIRE_ETL/INSPIRE_VISIT_DETAIL.csv', index=False)
df_visit_detail.to_parquet('INSPIRE_ETL/INSPIRE_VISIT_DETAIL.parquet')

In [804]:
df_visit_detail

,VISIT_DETAIL_ID,PERSON_ID,LOG_ID,VISIT_DETAIL_CONCEPT_ID,VISIT_DETAIL_START_DATETIME,VISIT_DETAIL_START_DATE,VISIT_DETAIL_END_DATETIME,VISIT_DETAIL_END_DATE,VISIT_DETAIL_TYPE_CONCEPT_ID,PRECEDING_VISIT_DETAIL_ID,VISIT_OCCURRENCE_ID
0,4000001,1000002,fc53c06ee2cfe438,32037,2019-09-27 09:02:00,2019-09-27,2019-10-18 18:13:00,2019-10-18,32817,<NA>,3000003
1,4000002,1000002,542843d28ab06693,32037,2019-09-27 09:02:00,2019-09-27,2019-10-18 18:13:00,2019-10-18,32817,<NA>,3000004
2,4000003,1000002,5b494bfd62a3afac,32037,2019-09-27 09:02:00,2019-09-27,2019-10-18 18:13:00,2019-10-18,32817,<NA>,3000005
3,4000004,1000002,6616be49ea19388b,32037,2019-09-27 09:02:00,2019-09-27,2019-10-18 18:13:00,2019-10-18,32817,<NA>,3000006
4,4000005,1000002,dbecca39feecd9ae,32037,2018-07-26 05:36:00,2018-07-26,2018-07-28 16:33:00,2018-07-28,32817,<NA>,3000007
...,...,...,...,...,...,...,...,...,...,...,...
29126,4029127,1039661,eb532568a30d1524,32037,2021-10-01 14:38:00,2021-10-01,2021-10-02 18:58:00,2021-10-02,32817,<NA>,3061910
29127,4029128,1039664,b2824373879ec86f,32037,2020-10-17 22:17:00,2020-10-17,2020-10-22 17:35:00,2020-10-22,32817,<NA>,3061911
29128,4029129,1039672,a060a15c8b5f3af8,32037,2022-03-31 23:44:00,2022-03-31,2022-04-03 22:42:00,2022-04-03,32817,<NA>,3061912
29129,4029130,1039679,d1177a1ac19137e2,32037,2022-10-31 18:54:00,2022-10-31,2022-11-04 19:07:00,2022-11-04,32817,<NA>,3061913


# CONDITION_OCCURRENCE

In [812]:
# Create an empty DataFrame using the predefined column names
df_cond_occ = pd.DataFrame(columns=['CONDITION_OCCURRENCE_ID'])

# Map PERSON_ID values from the df_person DataFrame to the new CONDITION_OCCURRENCE DataFrame
df_cond_occ['PERSON_ID'] = df_person['PERSON_ID']

# Transfer subject_id values (stored as PERSON_SOURCE_VALUE) from df_person to df_cond_occ
df_cond_occ['MRN'] = df_person['PERSON_SOURCE_VALUE']

# Merge the df_cond_occ DataFrame with df_diag using the 'subject_id' as a common column
df_cond_occ = df_cond_occ.merge(df_visit, on = 'MRN', how = 'left')

# Generate unique IDs for each row in the CONDITION_OCCURRENCE table
df_cond_occ['CONDITION_OCCURRENCE_ID'] = start_index['condition_occurrence'] + np.arange(len(df_cond_occ)) + 1

# Rename columns to match the target column names in the CONDITION_OCCURRENCE table
df_cond_occ.rename(columns={'concept_id': 'CONDITION_CONCEPT_ID', 
                            'dx_name': 'CONDITION_SOURCE_VALUE', 
                            'diagnosis_code': 'CONDITION_SOURCE_CONCEPT_ID'}, inplace=True)

usecols = ['LOG_ID', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME', 'PATIENT_CLASS_GROUP']
df_cond_occ = df_cond_occ.merge(df_info[usecols], on='LOG_ID', how='left')

# Set condition_start_date as hospital admission time since there is no condition start_time.
df_cond_occ['CONDITION_START_DATETIME'] = pd.to_datetime(df_cond_occ['HOSP_ADMSN_TIME'])
df_cond_occ['CONDITION_START_DATE'] = pd.to_datetime(df_cond_occ['CONDITION_START_DATETIME'].dt.date)

# Set condition_end_date as hospital discharge time since there is no condition_end_time.
df_cond_occ['CONDITION_END_DATETIME'] = pd.to_datetime(df_cond_occ['HOSP_DISCH_TIME'])
df_cond_occ['CONDITION_END_DATE'] = pd.to_datetime(df_cond_occ['CONDITION_END_DATETIME'].dt.date)

# Assign the CONDITION_TYPE_CONCEPT_ID indicating the data source is an EHR since it is not specified
df_cond_occ['CONDITION_TYPE_CONCEPT_ID'] = 32817

## Match visit_occurrence_id, visit_detail based on LOG_ID
df_cond_occ = df_cond_occ.merge(df_visit_occ[['LOG_ID', 'VISIT_OCCURRENCE_ID']], on='LOG_ID', how='left')
df_cond_occ = df_cond_occ.merge(df_visit_detail[['LOG_ID', 'VISIT_DETAIL_ID']], on='LOG_ID', how='left')

# Remove columns that aren't part of the CONDITION_OCCURRENCE table format
df_cond_occ.drop(columns=usecols, inplace=True)
df_cond_occ = df_cond_occ.astype({'VISIT_OCCURRENCE_ID':'Int64', 'VISIT_DETAIL_ID':'Int64'})

# Save the final df_cond_occ DataFrame to a CSV file
df_cond_occ.to_csv(f'{save_path}/MOVER_CONDITION_OCCURRENCE.csv', index=False)

In [815]:
df_cond_occ

,CONDITION_OCCURRENCE_ID,PERSON_ID,MRN,CONDITION_SOURCE_CONCEPT_ID,CONDITION_SOURCE_VALUE,CONDITION_CONCEPT_ID,CONDITION_START_DATETIME,CONDITION_START_DATE,CONDITION_END_DATETIME,CONDITION_END_DATE,CONDITION_TYPE_CONCEPT_ID,VISIT_OCCURRENCE_ID,VISIT_DETAIL_ID
0,5000001,1000001,2e92522a5f2a2a22,NaN,Pelvic organ prolapse quantification stage 2 c...,<NA>,2018-12-20 11:20:00,2018-12-20,2018-12-21 12:15:00,2018-12-21,32817,3000001,<NA>
1,5000002,1000001,2e92522a5f2a2a22,626.9,Abnormal uterine bleeding,44827912,2018-12-20 11:20:00,2018-12-20,2018-12-21 12:15:00,2018-12-21,32817,3000001,<NA>
2,5000003,1000001,2e92522a5f2a2a22,624.3,Hypertrophy of labia,44819837,2018-12-20 11:20:00,2018-12-20,2018-12-21 12:15:00,2018-12-21,32817,3000001,<NA>
3,5000004,1000001,2e92522a5f2a2a22,611.0,Cellulitis of breast,44831340,2021-02-03 18:51:00,2021-02-03,2021-02-06 20:00:00,2021-02-06,32817,3000002,<NA>
4,5000005,1000002,b7a91e623d957d8f,NaN,Respiratory failure requiring intubation (CMS-...,<NA>,2019-09-27 09:02:00,2019-09-27,2019-10-18 18:13:00,2019-10-18,32817,3000006,4000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134986,5121581,1039684,aecf54e7e0077c0b,276.2,Lactic acidosis,44828808,2021-06-07 03:35:00,2021-06-07,2021-08-03 13:00:00,2021-08-03,32817,3065727,4029131
134987,5121582,1039684,aecf54e7e0077c0b,401.9,"Hypertensive emergency, no CHF",44821949,2021-06-07 03:35:00,2021-06-07,2021-08-03 13:00:00,2021-08-03,32817,3065727,4029131
134988,5121583,1039684,aecf54e7e0077c0b,305.90,Polysubstance abuse (CMS-HCC),44831107,2021-06-07 03:35:00,2021-06-07,2021-08-03 13:00:00,2021-08-03,32817,3065727,4029131
134989,5121584,1039685,89d24047a41d7048,592.0,Kidney stone on left side,44826732,2022-09-06 10:25:00,2022-09-06,2022-09-06 22:50:00,2022-09-06,32817,3065728,<NA>


# DRUG_EXPOSURE

In [15]:
df1 = df_medi[(df_medi['RECORD_TYPE']=='INTRA-OP')] #& (df_medi['MEDICATION_NM'].str.contains('PROPOFOL'))]
df1.drop_duplicates(subset=['MRN', 'MEDICATION_NM'], inplace=True)
result = df1['MEDICATION_NM'].value_counts().to_frame()
result['perc'] = result['count'] / 39685 * 100
result.to_csv('mover_intraop_medication.csv')

/tmp/ipykernel_1911732/2079876851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.drop_duplicates(subset=['MRN', 'MEDICATION_NM'], inplace=True)


In [21]:
df_person = pd.read_csv(f'{save_path}/MOVER_PERSON.csv')

In [28]:
# Create an empty DataFrame using the predefined column names
df_drug = pd.DataFrame(columns = ['DRUG_EXPOSURE_ID'])

# Copy PERSON_ID values from the PERSON table to the DRUG_EXPOSURE table
df_drug['PERSON_ID'] = df_person['PERSON_ID']
# Copy PERSON_SOURCE_VALUE values as MRN from df_person to df_drug
df_drug['MRN'] = df_person['PERSON_SOURCE_VALUE']

usecols = ['LOG_ID', 'MRN', 'MEDICATION_ID', 'MEDICATION_NM', 'START_DATE', 'END_DATE', 'RECORD_TYPE', 'ADMIN_SIG', 'DOSE_UNIT_NM', 'MED_ROUTE_NM', 'concept_id', 'unit_concept_id']
df_drug = df_drug.merge(df_medi[usecols], on='MRN', how='left')

# Map 'concept_id' values to 'DRUG_CONCEPT_ID' column
df_drug['DRUG_CONCEPT_ID'] = df_drug['concept_id']
df_drug.dropna(subset='DRUG_CONCEPT_ID', inplace=True)

# Assign unique sequential IDs to the 'DRUG_EXPOSURE_ID' column
df_drug['DRUG_EXPOSURE_ID'] = start_index['drug_exposure'] + np.arange(len(df_drug)) + 1

# Convert 'chart_time' values (in minutes) to dates using the reference base_date
df_drug['DRUG_EXPOSURE_START_DATETIME'] = df_drug['START_DATE']
df_drug['DRUG_EXPOSURE_START_DATE'] = pd.to_datetime(df_drug['DRUG_EXPOSURE_START_DATETIME'].dt.date)
df_drug['DRUG_EXPOSURE_END_DATETIME'] = df_drug['END_DATE']
df_drug['DRUG_EXPOSURE_END_DATE'] = pd.to_datetime(df_drug['DRUG_EXPOSURE_END_DATETIME'].dt.date)
df_drug['VERBATIM_END_DATE'] = df_drug['DRUG_EXPOSURE_END_DATE']

# Assign 32838 (EHR Episode Record) for Intra-Op record, and 32817 (EHR) for others (Post-Op, Pre-Op).
df_drug['DRUG_TYPE_CONCEPT_ID'] = df_drug['RECORD_TYPE'].map({'INTRA-OP':32838})
df_drug.loc[df_drug['DRUG_TYPE_CONCEPT_ID'].isna(), 'DRUG_TYPE_CONCEPT_ID'] = 32817

# Map drug quantity values from 'ADMIN_SIG' column
df_drug['QUANTITY'] = df_drug['ADMIN_SIG']

# Map drug administration route to corresponding Standard Concept IDs
df_drug['ROUTE_CONCEPT_ID'] = df_drug['MED_ROUTE_NM'].map({'Oral': 4132161, 'IntraVENOUS': 4171047, 'Topical': 4263689}, na_action='ignore')
               
## Match visit_occurrence_id, visit_detail based on LOG_ID
df_drug = df_drug.merge(df_visit_occ[['LOG_ID', 'VISIT_OCCURRENCE_ID']], on='LOG_ID', how='left')
df_drug = df_drug.merge(df_visit_detail[['LOG_ID', 'VISIT_DETAIL_ID']], on='LOG_ID', how='left')
    
# Map drug names to the 'DRUG_SOURCE_VALUE' column
df_drug['DRUG_SOURCE_VALUE'] = df_drug['MEDICATION_NM']
df_drug['DRUG_SOURCE_CONCEPT_ID'] = df_drug['MEDICATION_ID']

# Map drug routes to the 'ROUTE_SOURCE_VALUE' column
df_drug['ROUTE_SOURCE_VALUE'] = df_drug['MED_ROUTE_NM']

df_drug['DOSE_UNIT_SOURCE_VALUE'] = df_drug['unit_concept_id']

# Filter the columns in df_drug to keep only the originally defined columns
df_drug.drop(columns=usecols, inplace=True)

# Save the final df_drug DataFrame to a CSV file
df_drug.to_csv(f'{save_path}/MOVER_DRUG_EXPOSURE.csv', index=False)
df_drug.to_parquet(f'{save_path}/MOVER_DRUG_EXPOSURE.parquet')

In [29]:
df_drug

,DRUG_EXPOSURE_ID,PERSON_ID,DRUG_CONCEPT_ID,DRUG_EXPOSURE_START_DATETIME,DRUG_EXPOSURE_START_DATE,DRUG_EXPOSURE_END_DATETIME,DRUG_EXPOSURE_END_DATE,VERBATIM_END_DATE,DRUG_TYPE_CONCEPT_ID,QUANTITY,ROUTE_CONCEPT_ID,VISIT_OCCURRENCE_ID,VISIT_DETAIL_ID,DRUG_SOURCE_VALUE,DRUG_SOURCE_CONCEPT_ID,ROUTE_SOURCE_VALUE,DOSE_UNIT_SOURCE_VALUE
0,6000001,1000001,1771162,2018-12-20,2018-12-20,2018-12-21,2018-12-21,2018-12-21,32817.0,NaN,4171047.0,3000001.0,NaN,CEFAZOLIN SODIUM-DEXTROSE 2-4 GM/100ML-% IV SOLN,118010.0,IntraVENOUS,8576
1,6000002,1000001,1124957,2018-12-20,2018-12-20,2018-12-21,2018-12-21,2018-12-21,32817.0,10.0,4132161.0,3000001.0,NaN,OXYCODONE HCL 5 MG OR TABS,10814.0,Oral,8576
2,6000003,1000001,915981,2018-12-20,2018-12-20,2018-12-20,2018-12-20,2018-12-20,32838.0,1000.0,NaN,3000001.0,NaN,NEOMYCIN-POLYMIXIN GU IRRIGANT 1 ML IN 1000 ML...,452691.0,Irrigation,<NA>
3,6000004,1000001,1125315,2018-12-20,2018-12-20,2018-12-21,2018-12-21,2018-12-21,32817.0,1000.0,4132161.0,3000001.0,NaN,ACETAMINOPHEN 500 MG OR TABS,102.0,Oral,8576
4,6000005,1000001,1149380,2018-12-20,2018-12-20,2018-12-21,2018-12-21,2018-12-21,32817.0,2.0,NaN,3000001.0,NaN,FLUTICASONE PROPIONATE HFA 220 MCG/ACT IN AERO,41205.0,Inhalation,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25649111,30637420,1039685,1124957,2022-09-06,2022-09-06,2022-09-07,2022-09-07,2022-09-07,32817.0,10.0,4132161.0,3065728.0,NaN,OXYCODONE HCL 5 MG OR TABS,10814.0,Oral,8576
25649112,30637421,1039685,989878,2022-09-06,2022-09-06,2022-09-06,2022-09-06,2022-09-06,32838.0,40.0,4171047.0,3065728.0,NaN,LIDOCAINE 1% SOLN OPTIME (UCI),509260.0,IntraVENOUS,8576
25649113,30637422,1039685,753626,2022-09-06,2022-09-06,2022-09-06,2022-09-06,2022-09-06,32838.0,30.0,4171047.0,3065728.0,NaN,PROPOFOL 200 MG/20ML IV EMUL,119103.0,IntraVENOUS,8576
25649114,30637423,1039685,1135766,2022-09-06,2022-09-06,2022-09-06,2022-09-06,2022-09-06,32838.0,100.0,4171047.0,3065728.0,NaN,PHENYLEPHRINE DILUTION 100 MCG/ML IJ SOLN,500840.0,IntraVENOUS,9655


# PROCEDURE_OCCURRENCE

In [59]:
# Initialize an empty DataFrame for the PROCEDURE_OCCURRENCE table using the specified columns
df_proc = pd.DataFrame(columns=['PROCEDURE_CONCEPT_ID'])

# Map corresponding PERSON_ID values from the PERSON table to the new PROCEDURE_OCCURRENCE DataFrame
df_proc['PERSON_ID'] = df_person['PERSON_ID']

# Extract MRN from PERSON_SOURCE_VALUE for mapping with the operation data
df_proc['MRN'] = df_person['PERSON_SOURCE_VALUE']

# Merge information table with selected columns with procedure occurrence table based on MRN
usecols = ['LOG_ID', 'MRN', 'PRIMARY_PROCEDURE_NM', 'IN_OR_DTTM', 'OUT_OR_DTTM', 'concept_id']
df_proc = df_proc.merge(df_info, on='MRN', how='left')

# Generate unique identifiers for each procedure occurrence
df_proc['PROCEDURE_OCCURRENCE_ID'] = start_index['procedure_occurrence_id'] + np.arange(len(df_proc)) + 1

# Assign the standard concept IDs to the procedure concept ID column
df_proc['PROCEDURE_CONCEPT_ID'] = df_proc['standard_concept_id']


df_proc['PROCEDURE_DATETIME'] = pd.to_datetime(df_proc['IN_OR_DTTM'])
df_proc['PROCEDURE_DATE'] = pd.to_datetime(df_proc['PROCEDURE_DATETIME'].dt.date)
df_proc['PROCEDURE_END_DATETIME'] = pd.to_datetime(df_proc['OUT_OR_DTTM'])
df_proc['PROCEDURE_END_DATE'] = pd.to_datetime(df_proc['PROCEDURE_END_DATETIME'].dt.date)

# Assign a type concept ID indicating the data is sourced from an EHR Episode Record since it is intra-op record.
df_proc['PROCEDURE_TYPE_CONCEPT_ID'] = 32838

# Link each procedure to a corresponding visit by merging with the visit occurrence data
df_proc['VISIT_OCCURRENCE_ID'] = df_proc.merge(df_visit_occ[['hadm_id', 'VISIT_OCCURRENCE_ID']], 
                                               on='hadm_id', suffixes=('_x', None), how='left')['VISIT_OCCURRENCE_ID']
df_proc['VISIT_DETAIL_ID'] = df_proc.merge(df_visit_detail[['hadm_id', 'VISIT_DETAIL_ID']], 
                                                on='hadm_id', suffixes=('_x', None), how='left')['VISIT_DETAIL_ID']

# Populate source value and source concept ID columns using the operation data
df_proc['PROCEDURE_SOURCE_VALUE'] = df_proc['PRIMARY_PROCEDURE_NM']
#df_proc['PROCEDURE_SOURCE_CONCEPT_ID'] = df_proc['source_concept_id']

# Filter the DataFrame to only include the originally defined columns
df_proc = df_proc[columns]

# Export the final PROCEDURE_OCCURRENCE data to a CSV file
df_proc.to_csv(f'{save_path}/MOVER_PROCEDURE_OCCURRENCE.csv', index=False)

In [39]:
df_labs.loc[df_labs['Lab Name']=='Base excess^^standard', 'Observation Value'].value_counts()

Observation Value
9999999.0    33196
0.0           6627
1.0           5960
2.0           5067
3.0           3802
4.0           3183
5.0           2327
6.0           1613
7.0           1107
8.0            743
9.0            617
11.0           343
10.0           342
12.0           213
13.0           119
14.0            79
15.0            62
16.0            60
17.0            57
18.0            32
20.0            20
19.0            10
25.0             8
26.0             3
23.0             3
27.0             3
30.0             2
21.0             2
22.0             1
Name: count, dtype: int64

In [36]:
df_labs[df_labs['Lab Name'].str.contains('Base excess^^standard')]

,LOG_ID,MRN,Lab Code,Lab Name,Observation Value,Measurement Units,Reference Range,Abnormal Flag,Collection Datetime,concept_id,unit_concept_id


# MEASUREMENT

In [11]:
df_measure = pd.DataFrame(columns = ['MEASUREMENT_ID'])

# Copy PERSON_ID values from the PERSON table to the MEASUREMENT table
df_measure['PERSON_ID'] = df_person['PERSON_ID']
# Copy PERSON_SOURCE_VALUE values as subject_id from df_person to df_measure
df_measure['subject_id'] = df_person['PERSON_SOURCE_VALUE']

# Filter out rows in df_vitals with null 'vocab' values
df_v = df_vitals.dropna(subset='vocab')
df_v = df_v.loc[df_v['vocab']=='LOINC'].drop(columns=['op_id', 'vocab'])

print(f'line34 : {time.time() - start}')
start = time.time()
# Concatenate df_v and df_m vertically
records = pd.concat([df_labs, df_v, df_ward], axis = 0)
records = records.merge(df_params[['Unit', 'unit_concept_id']].drop_duplicates(subset='Unit'), on='Unit', how='left')
del df_v

df_measure = df_measure.merge(records, on='subject_id', how='left')
del records

print(f'line44 : {time.time() - start}')
start = time.time()

# Generate unique identifiers for each measurement
df_measure['MEASUREMENT_ID'] = np.arange(len(df_measure)) + 1

df_measure['MEASUREMENT_CONCEPT_ID'] = df_measure['concept_id']

base_date = datetime(2011, 1, 1)
df_measure['MEASUREMENT_DATETIME'] = base_date + pd.to_timedelta(df_measure['chart_time'], unit='min')
df_measure['MEASUREMENT_DATE'] = df_measure['MEASUREMENT_DATETIME'].dt.date

# Assign a type concept ID indicating the data is sourced from an EHR
df_measure['MEASUREMENT_TYPE_CONCEPT_ID'] = 32817

# Concept for '=' operation
df_measure['OPERATOR_CONCEPT_ID'] = 4172703

print(f'line62 : {time.time() - start}')
start = time.time()
#df_measure['VALUE_AS_NUMBER'] = df_measure['value']
exceptions = [3003396, 3002032, 3006277, 3012501, 3003129, 3004959, 3007435]
valid_mask = (df_measure['value'] >= 0) | (df_measure['concept_id'].isin(exceptions))
df_measure.loc[valid_mask, 'VALUE_AS_NUMBER'] = df_measure['value']
df_measure.loc[~valid_mask, 'VALUE_AS_NUMBER'] = None

print(f'line70 : {time.time() - start}')
start = time.time()
df_measure['UNIT_CONCEPT_ID'] = df_measure['unit_concept_id']

# Convert 'chart_time' values (in minutes) again for merging with visit occurrences
df_measure['chart_date'] = base_date + pd.to_timedelta(df_measure['chart_time'], unit='min')

# Match measure exposure dates with visit occurrences based on 'PERSON_ID'
result = pd.merge(df_measure[['PERSON_ID', 'chart_date', 'MEASUREMENT_ID']], 
                df_visit_occ, on='PERSON_ID', how='left')
# Filter results to keep only those rows where 'chart_date' falls within a visit's start and end times
result = result[(result['chart_date'] >= result['VISIT_START_DATETIME']) & 
                (result['chart_date'] <= result['VISIT_END_DATETIME'])]

# Merge the filtered results with df_cond_occ to add 'VISIT_OCCURRENCE_ID' details to df_measure
df_measure['VISIT_OCCURRENCE_ID'] =  df_measure.merge(result[['MEASUREMENT_ID', 'VISIT_OCCURRENCE_ID']], 
                                on='MEASUREMENT_ID', 
                                how='left', 
                                suffixes=('_x', None))['VISIT_OCCURRENCE_ID']


# Link measurements to their associated visit detail
linked_visits = pd.merge(df_measure[['PERSON_ID', 'chart_date', 'MEASUREMENT_ID']], df_visit_detail, on='PERSON_ID', how='left')
# Filter to include only measurements taken during a visit's duration
linked_visits = linked_visits[(linked_visits['chart_date'] >= linked_visits['VISIT_DETAIL_START_DATETIME']) & 
                              (linked_visits['chart_date'] <= linked_visits['VISIT_DETAIL_END_DATETIME'])]
df_measure['VISIT_DETAIL_ID'] = df_measure.merge(linked_visits[['MEASUREMENT_ID', 'VISIT_DETAIL_ID']], 
                                                     on='MEASUREMENT_ID', how='left',suffixes=('_x', None))['VISIT_DETAIL_ID']

del linked_visits

print(f'line90 : {time.time() - start}')
start = time.time()
#df_measure['VISIT_DETAIL_ID'] = df_measure['VISIT_OCCURRENCE_ID']

df_measure['MEASUREMENT_SOURCE_VALUE'] = df_measure['value']

df_measure['UNIT_SOURCE_VALUE'] = df_measure['Unit']

df_measure = df_measure[columns]

df_measure.to_csv('INSPIRE_ETL/INSPIRE_MEASUREMENT.csv', index=False)

print(f'line102 : {time.time() - start}')

start = time.time()

line34 : 10.768936157226562
line44 : 71.09569549560547
line62 : 26.593946933746338
line70 : 46.20182418823242
line90 : 174.40139317512512
line102 : 887.9683330059052


In [6]:
start = time.time()

# Define the columns for the MEASUREMENT table in OMOP CDM format
columns = ['MEASUREMENT_ID',
 'PERSON_ID',
 'MEASUREMENT_CONCEPT_ID',
 'MEASUREMENT_DATE',
 'MEASUREMENT_DATETIME',
 'MEASUREMENT_TIME',
 'MEASUREMENT_TYPE_CONCEPT_ID',
 'OPERATOR_CONCEPT_ID',
 'VALUE_AS_NUMBER',
 'VALUE_AS_CONCEPT_ID',
 'UNIT_CONCEPT_ID',
 'RANGE_LOW',
 'RANGE_HIGH',
 'PROVIDER_ID',
 'VISIT_OCCURRENCE_ID',
 'VISIT_DETAIL_ID',
 'MEASUREMENT_SOURCE_VALUE',
 'MEASUREMENT_SOURCE_CONCEPT_ID',
 'UNIT_SOURCE_VALUE',
 'VALUE_SOURCE_VALUE']

# Initialize an empty DataFrame for storing MEASUREMENT data
df_measure = pd.DataFrame(columns=columns)

# Populate 'PERSON_ID' and 'subject_id' columns in MEASUREMENT table from the PERSON table
df_measure['PERSON_ID'] = df_person['PERSON_ID']
df_measure['subject_id'] = df_person['PERSON_SOURCE_VALUE']

# Filter out measurements from df_vitals where the vocabulary is not LOINC
df_v = df_vitals.dropna(subset='vocab')
df_v = df_v[df_v['vocab']=='LOINC'].drop(['op_id', 'vocab'], axis=1)

# Combine data from various sources (labs, vitals, wards) into a single DataFrame
records = pd.concat([df_labs, df_v, df_ward], axis=0)
# Enrich the combined records with associated unit concept IDs
records = records.merge(df_params[['Unit', 'unit_concept_id']].drop_duplicates(subset='Unit'), on='Unit', how='left')
# Release memory
del df_v

# Merge the enriched records with MEASUREMENT table on 'subject_id'
df_measure = df_measure.merge(records, on='subject_id', how='left')
# Release memory
del records

# Assign unique MEASUREMENT_IDs to each row
df_measure['MEASUREMENT_ID'] = np.arange(1, len(df_measure) + 1)

# Extract and set the relevant concept and datetime details for each measurement
df_measure['MEASUREMENT_CONCEPT_ID'] = df_measure['concept_id']
base_date = datetime(2011, 1, 1)
df_measure['MEASUREMENT_DATETIME'] = base_date + pd.to_timedelta(df_measure['chart_time'], unit='min')
df_measure['MEASUREMENT_DATE'] = df_measure['MEASUREMENT_DATETIME'].dt.date
df_measure['MEASUREMENT_TYPE_CONCEPT_ID'] = 32817  # EHR as data source
df_measure['OPERATOR_CONCEPT_ID'] = 4172703  # '=' operation

# Handle special cases for 'VALUE_AS_NUMBER' based on specific concept IDs
exceptions = [3003396, 3002032, 3006277, 3012501, 3003129, 3004959, 3007435]
valid_mask = (df_measure['value'] >= 0) | df_measure['concept_id'].isin(exceptions)
df_measure.loc[valid_mask, 'VALUE_AS_NUMBER'] = df_measure['value']
df_measure.loc[~valid_mask, 'VALUE_AS_NUMBER'] = None

# Set the 'UNIT_CONCEPT_ID' values
df_measure['UNIT_CONCEPT_ID'] = df_measure['unit_concept_id']

## Match visit_occurrence_id, visit_detail_id based on chart_time
# Convert 'chart_time' to 'chart_date' to facilitate visit occurrence matching
df_measure['chart_date'] = base_date + pd.to_timedelta(df_measure['chart_time'], unit='min')

# Link measurements to their associated visit occurrences
linked_visits = pd.merge(df_measure[['PERSON_ID', 'chart_date', 'MEASUREMENT_ID']], df_visit_occ, on='PERSON_ID', how='left')
# Filter to include only measurements taken during a visit's duration
linked_visits = linked_visits[(linked_visits['chart_date'] >= linked_visits['VISIT_START_DATETIME']) & 
                              (linked_visits['chart_date'] <= linked_visits['VISIT_END_DATETIME'])]
df_measure['VISIT_OCCURRENCE_ID'] = df_measure.merge(linked_visits[['MEASUREMENT_ID', 'VISIT_OCCURRENCE_ID']], 
                                                     on='MEASUREMENT_ID', how='left', suffixes=('_x', None))['VISIT_OCCURRENCE_ID']


# Link measurements to their associated visit detail
linked_visits = pd.merge(df_measure[['PERSON_ID', 'chart_date', 'MEASUREMENT_ID']], df_visit_detail, on='PERSON_ID', how='left')
# Filter to include only measurements taken during a visit's duration
linked_visits = linked_visits[(linked_visits['chart_date'] >= linked_visits['VISIT_DETAIL_START_DATETIME']) & 
                              (linked_visits['chart_date'] <= linked_visits['VISIT_DETAIL_END_DATETIME'])]
df_measure['VISIT_DETAIL_ID'] = df_measure.merge(linked_visits[['MEASUREMENT_ID', 'VISIT_DETAIL_ID']], 
                                                     on='MEASUREMENT_ID', how='left',suffixes=('_x', None))['VISIT_DETAIL_ID']

del linked_visits

# Set source value columns
df_measure['MEASUREMENT_SOURCE_VALUE'] = df_measure['value']
df_measure['UNIT_SOURCE_VALUE'] = df_measure['Unit']

# Retain only the relevant columns in the final MEASUREMENT table
df_measure = df_measure[columns]

print(time.time() - start)
start = time.time()

# Export the final MEASUREMENT table to CSV
df_measure.to_csv('INSPIRE_ETL/INSPIRE_MEASUREMENT.csv', index=False)

print(time.time() - start)

984.7650737762451
1737.8386561870575


In [9]:
df_measure[~df_measure['VISIT_DETAIL_ID'].isna()]

,MEASUREMENT_ID,PERSON_ID,MEASUREMENT_CONCEPT_ID,MEASUREMENT_DATE,MEASUREMENT_DATETIME,MEASUREMENT_TIME,MEASUREMENT_TYPE_CONCEPT_ID,OPERATOR_CONCEPT_ID,VALUE_AS_NUMBER,VALUE_AS_CONCEPT_ID,UNIT_CONCEPT_ID,RANGE_LOW,RANGE_HIGH,PROVIDER_ID,VISIT_OCCURRENCE_ID,VISIT_DETAIL_ID,MEASUREMENT_SOURCE_VALUE,MEASUREMENT_SOURCE_CONCEPT_ID,UNIT_SOURCE_VALUE,VALUE_SOURCE_VALUE
549,550,2,3014111.0,2011-01-03,2011-01-03 15:20:00,NaN,32817,4172703,2.6,NaN,8753.0,NaN,NaN,NaN,2.0,1.0,2.60,NaN,mmol/L,NaN
550,551,2,3014111.0,2011-01-06,2011-01-06 09:05:00,NaN,32817,4172703,1.9,NaN,8753.0,NaN,NaN,NaN,2.0,1.0,1.90,NaN,mmol/L,NaN
552,553,2,3016723.0,2011-01-03,2011-01-03 19:05:00,NaN,32817,4172703,2.76,NaN,8840.0,NaN,NaN,NaN,2.0,1.0,2.76,NaN,mg/dL,NaN
553,554,2,3016723.0,2011-01-03,2011-01-03 23:15:00,NaN,32817,4172703,2.41,NaN,8840.0,NaN,NaN,NaN,2.0,1.0,2.41,NaN,mg/dL,NaN
554,555,2,3016723.0,2011-01-04,2011-01-04 04:20:00,NaN,32817,4172703,2.49,NaN,8840.0,NaN,NaN,NaN,2.0,1.0,2.49,NaN,mg/dL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127393840,127393841,99893,21490528.0,2011-01-02,2011-01-02 14:40:00,NaN,32817,4172703,34.0,NaN,8554.0,NaN,NaN,NaN,NaN,14209.0,34.00,NaN,%,NaN
127393841,127393842,99893,3024171.0,2011-01-02,2011-01-02 14:40:00,NaN,32817,4172703,12.0,NaN,8541.0,NaN,NaN,NaN,NaN,14209.0,12.00,NaN,/min,NaN
127393842,127393843,99893,3027018.0,2011-01-02,2011-01-02 14:45:00,NaN,32817,4172703,66.0,NaN,8541.0,NaN,NaN,NaN,NaN,14209.0,66.00,NaN,/min,NaN
127393843,127393844,99893,3013502.0,2011-01-02,2011-01-02 14:45:00,NaN,32817,4172703,99.0,NaN,8554.0,NaN,NaN,NaN,NaN,14209.0,99.00,NaN,%,NaN


In [7]:
len(df_measure)

127398829

In [8]:
df_measure[:100]

,MEASUREMENT_ID,PERSON_ID,MEASUREMENT_CONCEPT_ID,MEASUREMENT_DATE,MEASUREMENT_DATETIME,MEASUREMENT_TIME,MEASUREMENT_TYPE_CONCEPT_ID,OPERATOR_CONCEPT_ID,VALUE_AS_NUMBER,VALUE_AS_CONCEPT_ID,UNIT_CONCEPT_ID,RANGE_LOW,RANGE_HIGH,PROVIDER_ID,VISIT_OCCURRENCE_ID,VISIT_DETAIL_ID,MEASUREMENT_SOURCE_VALUE,MEASUREMENT_SOURCE_CONCEPT_ID,UNIT_SOURCE_VALUE,VALUE_SOURCE_VALUE
0,1,1,3018677.0,2011-01-01,2011-01-01 15:25:00,NaN,32817,4172703,36.7,NaN,8555.0,NaN,NaN,NaN,1.0,NaN,36.70,NaN,sec,NaN
1,2,1,3034426.0,2011-01-01,2011-01-01 15:25:00,NaN,32817,4172703,1.05,NaN,44818586.0,NaN,NaN,NaN,1.0,NaN,1.05,NaN,INR,NaN
2,3,1,3016407.0,2011-01-01,2011-01-01 15:25:00,NaN,32817,4172703,222.0,NaN,8840.0,NaN,NaN,NaN,1.0,NaN,222.00,NaN,mg/dL,NaN
3,4,1,3000963.0,2011-01-01,2011-01-01 15:25:00,NaN,32817,4172703,8.7,NaN,8713.0,NaN,NaN,NaN,1.0,NaN,8.70,NaN,g/dL,NaN
4,5,1,3009542.0,2011-01-01,2011-01-01 15:25:00,NaN,32817,4172703,30.5,NaN,8554.0,NaN,NaN,NaN,1.0,NaN,30.50,NaN,%,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1,3020891.0,2011-01-01,2011-01-01 18:40:00,NaN,32817,4172703,28.6,NaN,586323.0,NaN,NaN,NaN,1.0,NaN,28.60,NaN,Celsius,NaN
96,97,1,21490569.0,2011-01-01,2011-01-01 18:40:00,NaN,32817,4172703,19.0,NaN,8876.0,NaN,NaN,NaN,1.0,NaN,19.00,NaN,mmHg,NaN
97,98,1,21490624.0,2011-01-01,2011-01-01 18:40:00,NaN,32817,4172703,3.15,NaN,8554.0,NaN,NaN,NaN,1.0,NaN,3.15,NaN,%,NaN
98,99,1,21490718.0,2011-01-01,2011-01-01 18:40:00,NaN,32817,4172703,8.0,NaN,8698.0,NaN,NaN,NaN,1.0,NaN,8.00,NaN,L/min,NaN


# DEATH

In [13]:
import pandas as pd

# columns of PERSON
df_hira = pd.read_csv('sample/HIRA/HIRA_DEATH_v1.csv')
columns = list(df_hira.columns)
columns

['PERSON_ID',
 'DEATH_DATE',
 'DEATH_DATETIME',
 'DEATH_TYPE_CONCEPT_ID',
 'CAUSE_CONCEPT_ID',
 'CAUSE_SOURCE_VALUE',
 'CAUSE_SOURCE_CONCEPT_ID']

In [14]:
columns = ['PERSON_ID',
 'DEATH_DATE',
 'DEATH_DATETIME',
 'DEATH_TYPE_CONCEPT_ID',
 'CAUSE_CONCEPT_ID',
 'CAUSE_SOURCE_VALUE',
 'CAUSE_SOURCE_CONCEPT_ID']

# Initialize an empty DataFrame for storing DEATH table
df_death = pd.DataFrame(columns=columns)

# Populate 'PERSON_ID' and 'subject_id' columns in the DEATH table from the PERSON table
df_death['PERSON_ID'] = df_person['PERSON_ID']
df_death['subject_id'] = df_person['PERSON_SOURCE_VALUE']

# Merge 'inhosp_death_time' from the operations (df_op) table into the DEATH table using 'subject_id'
df_death = df_death.merge(df_op[['subject_id', 'inhosp_death_time']], on='subject_id', how='left')

# Define the reference date for datetime calculations
base_date = datetime(2011, 1, 1)

# Convert in-hospital death times to actual datetime objects using the base_date as the reference point
df_death['DEATH_DATETIME'] = base_date + pd.to_timedelta(df_death['inhosp_death_time'], unit='min')
df_death['DEATH_DATE'] = df_death['DEATH_DATETIME'].dt.date

# Set the DEATH_TYPE_CONCEPT_ID to represent data sourced from an Electronic Health Record (EHR)
df_death['DEATH_TYPE_CONCEPT_ID'] = 32817

# Retain only the relevant columns in the final DEATH table
df_death = df_death[columns]

# Export the final DEATH table to CSV
df_death.to_csv('INSPIRE_ETL/INSPIRE_DEATH.csv', index=False)

# NOTE

In [37]:
# Define the columns for the NOTE table in the OMOP CDM format
columns = [
    'NOTE_ID', 'PERSON_ID', 'NOTE_DATE', 'NOTE_DATETIME', 'NOTE_TYPE_CONCEPT_ID',
    'NOTE_CLASS_CONCEPT_ID', 'NOTE_TITLE', 'NOTE_TEXT', 'ENCODING_CONCEPT_ID',
    'LANGUAGE_CONCEPT_ID', 'PROVIDER_ID', 'VISIT_OCCURRENCE_ID', 'VISIT_DETAIL_ID',
    'NOTE_SOURCE_VALUE', 'NOTE_EVENT_ID', 'NOTE_EVENT_FIELD_CONCEPT_ID']

# Initialize an empty DataFrame for storing NOTE table
df_note = pd.DataFrame(columns=columns)

# Populate 'PERSON_ID' and 'subject_id' columns in the NOTE table from the PERSON table
df_note['PERSON_ID'] = df_person['PERSON_ID']
df_note['subject_id'] = df_person['PERSON_SOURCE_VALUE']


# residual fields that are not mapped in operation table
res_fields = ['asa', 'emop', 'department', 'antype', 'orin_time', 'orout_time', 'anstart_time', 'anend_time', 'cpbon_time', 'cpboff_time']
res_op = pd.melt(df_op, id_vars=['subject_id', 'opdate'], value_vars=res_fields)
df_note = df_note.merge(res_op, on='subject_id', how='left')
df_note.dropna(subset='value', inplace=True, ignore_index=True)

# Assign unique sequential IDs to the 'NOTE_ID' column
df_note['NOTE_ID'] = np.arange(len(df_note)) + 1

base_date = datetime(2011, 1, 1)
df_note['NOTE_DATETIME'] = base_date + pd.to_timedelta(df_note['opdate'], unit='min')
df_note['NOTE_DATE'] = df_note['NOTE_DATETIME'].dt.date

# Set the NOTE_TYPE_CONCEPT_ID to represent data sourced from an Electronic Health Record (EHR)
df_note['NOTE_TYPE_CONCEPT_ID'] = 32817

# Use the concept id  706617(Anesthesiology) or 706502(Surgical operation).
res_ane = ['asa', 'antype','anstart_time', 'anend_time']
df_note.loc[df_note['variable'].isin(res_ane), 'NOTE_CLASS_CONCEPT_ID'] = 706617
df_note.loc[~df_note['variable'].isin(res_ane), 'NOTE_CLASS_CONCEPT_ID'] = 706502

df_note['NOTE_TITLE'] = df_note['variable']
df_note['NOTE_TEXT'] = df_note['value']

# Use the concept_id 32678(UTF-8)
df_note['ENCODING_CONCEPT_ID'] = 32678

# Use the concept_id 4180186(English language)
df_note['LANGUAGE_CONCEPT_ID'] = 4180186 

df_note = match_visit(df_note, 'NOTE_ID', df_visit_occ, df_visit_detail, on = 'opdate')

# Retain only the relevant columns in the final NOTE table
df_note = df_note[columns]

# Export the final NOTE table to CSV
df_note.to_csv('INSPIRE_ETL/INSPIRE_NOTE.csv', index=False)

In [38]:
df_note

,NOTE_ID,PERSON_ID,NOTE_DATE,NOTE_DATETIME,NOTE_TYPE_CONCEPT_ID,NOTE_CLASS_CONCEPT_ID,NOTE_TITLE,NOTE_TEXT,ENCODING_CONCEPT_ID,LANGUAGE_CONCEPT_ID,PROVIDER_ID,VISIT_OCCURRENCE_ID,VISIT_DETAIL_ID,NOTE_SOURCE_VALUE,NOTE_EVENT_ID,NOTE_EVENT_FIELD_CONCEPT_ID
0,1,1,2011-01-01,2011-01-01,32817,706502,emop,1,32678,4180186,NaN,1.0,NaN,NaN,NaN,NaN
1,2,1,2011-01-01,2011-01-01,32817,706502,department,OT,32678,4180186,NaN,1.0,NaN,NaN,NaN,NaN
2,3,1,2011-01-01,2011-01-01,32817,706617,antype,General,32678,4180186,NaN,1.0,NaN,NaN,NaN,NaN
3,4,1,2011-01-01,2011-01-01,32817,706502,orin_time,1110,32678,4180186,NaN,1.0,NaN,NaN,NaN,NaN
4,5,1,2011-01-01,2011-01-01,32817,706502,orout_time,1245,32678,4180186,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049989,1049950,99900,2011-01-03,2011-01-03,32817,706617,antype,General,32678,4180186,NaN,126754.0,NaN,NaN,NaN,NaN
1049990,1049951,99900,2011-01-03,2011-01-03,32817,706502,orin_time,3355,32678,4180186,NaN,126754.0,NaN,NaN,NaN,NaN
1049991,1049952,99900,2011-01-03,2011-01-03,32817,706502,orout_time,3430,32678,4180186,NaN,126754.0,NaN,NaN,NaN,NaN
1049992,1049953,99900,2011-01-03,2011-01-03,32817,706617,anstart_time,3360.0,32678,4180186,NaN,126754.0,NaN,NaN,NaN,NaN


In [36]:
def match_visit(table, unique_id, df_visit_occ, df_visit_detail, on='chart_time'):
    # Matches visit_occurrence_id and visit_detail_id based on chart_time
    # table: a target table that needs to match visit_ids
    # unique_id: an unique identifier of a table
    # on: the column name for the reference
    
    # Convert 'chart_time' values (in minutes) again for merging with visit occurrences
    table['chart_date'] = base_date + pd.to_timedelta(table[on], unit='min')

    # Match drug exposure dates with visit occurrences based on 'PERSON_ID'
    result = pd.merge(table[['PERSON_ID', 'chart_date', unique_id]], 
                    df_visit_occ[['PERSON_ID', 'VISIT_OCCURRENCE_ID', 'VISIT_START_DATETIME', 'VISIT_END_DATETIME']], 
                      on='PERSON_ID', how='left')
    # Filter results to keep only those rows where 'chart_date' falls within a visit's start and end times
    result = result[(result['chart_date'] >= result['VISIT_START_DATETIME']) & 
                    (result['chart_date'] <= result['VISIT_END_DATETIME'])]

    
    # Merge the filtered results with df_cond_occ to add 'VISIT_OCCURRENCE_ID' details to table
    table = table.merge(result[[unique_id, 'VISIT_OCCURRENCE_ID']], 
                                    on=unique_id, 
                                    how='left', 
                                    suffixes=('_x', None))
    del result

    # Match drug exposure dates with visit occurrences based on 'PERSON_ID'
    result = pd.merge(table[['PERSON_ID', 'chart_date', unique_id]], 
                    df_visit_detail[['PERSON_ID', 'VISIT_DETAIL_ID', 'VISIT_DETAIL_START_DATETIME', 'VISIT_DETAIL_END_DATETIME']], 
                      on='PERSON_ID', how='left')
    # Filter results to keep only those rows where 'chart_date' falls within a visit's start and end times
    result = result[(result['chart_date'] >= result['VISIT_DETAIL_START_DATETIME']) & 
                    (result['chart_date'] <= result['VISIT_DETAIL_END_DATETIME'])]

    # Merge the filtered results with df_cond_occ to add 'VISIT_DETAIL_ID' details to table
    table = table.merge(result[[unique_id, 'VISIT_DETAIL_ID']], 
                                    on=unique_id, 
                                    how='left', 
                                    suffixes=('_x', None))  
    del result
    
    return table

# EDA of data

In [116]:
df_op[df_op['subject_id']==178742874]

,op_id,subject_id,hadm_id,case_id,opdate,age,sex,weight,height,race,...,opend_time,admission_time,discharge_time,anstart_time,anend_time,cpbon_time,cpboff_time,icuin_time,icuout_time,inhosp_death_time
0,484069807,178742874,229842382,NaN,0,30,F,48.0,153.0,Asian,...,1230.0,0,7195,1120.0,1235.0,NaN,NaN,NaN,NaN,NaN


In [118]:
df1 = df_vitals[df_vitals['subject_id']==178742874]
df1

,op_id,subject_id,chart_time,item_name,value,concept_id,vocab
22468646,484069807,178742874,1120,rr,12.0,3024171.0,LOINC
22468647,484069807,178742874,1125,rr,26.5,3024171.0,LOINC
22468648,484069807,178742874,1130,rr,15.0,3024171.0,LOINC
22468649,484069807,178742874,1135,rr,15.0,3024171.0,LOINC
22468650,484069807,178742874,1140,rr,15.0,3024171.0,LOINC
...,...,...,...,...,...,...,...
62482745,484069807,178742874,1240,etco2,38.0,21490569.0,LOINC
62482746,484069807,178742874,1240,minvol,4.4,42527120.0,LOINC
62482747,484069807,178742874,1240,vt,288.0,21490854.0,LOINC
62482748,484069807,178742874,1240,rr,28.0,3024171.0,LOINC


In [119]:
df1[df1['vocab'] == 'RxNorm']

,op_id,subject_id,chart_time,item_name,value,concept_id,vocab
62482396,484069807,178742874,1020,hs,0.0,40166953.0,RxNorm
62482594,484069807,178742874,1180,hs,300.0,40166953.0,RxNorm


In [120]:
df_labs[df_labs['subject_id']==178742874]

,subject_id,chart_time,item_name,value,concept_id
10668380,178742874,925,aptt,36.70,3018677
10668381,178742874,925,ptinr,1.05,3034426
10668382,178742874,925,fibrinogen,222.00,3016407
10668383,178742874,925,hb,8.70,3000963
10668384,178742874,925,hct,30.50,3009542
10668385,178742874,925,wbc,5.62,3010813
10668386,178742874,925,platelet,232.00,3007461
10668387,178742874,925,rbc,3.79,3040494
10668388,178742874,925,lymphocyte,27.20,3019198
10668389,178742874,925,sodium,138.00,3019550


# Final
* person table에 subject_id 제거